# Imports

In [1]:
from sklearn.utils import shuffle
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import joblib
import pandas as pd
import os
import sagemaker
import re

# Model Deploy

By using the artifacts created by the mode, SageMaker can deploy the model so it is a service running. 
For this part we will create an endpoint (service of the model running) from the predictor defined and fit before. Next, we will use a single tweet to process it and send it to the endpoint (the model deployed) and wait for an answer from it. 

## Processing of a single tweet

First, since we will process a single tweet, which should be a line of text, we have to pre process it. It is important to remember that the model hadels an matrix of numbers representing the text (word vectorization).

In [2]:
tweet = "@user why not @user mocked obama for being black.  @user @user @user @user #brexit" 
#this tweet is from the database, it is labelled as 1, which means its cathegorized as violent.

To transform this single tweet we should use the same function we used in the data exploration step. Since that function is defined in a different notebook, we will copy it and re define it here.

In [3]:
def tweet_to_words(tweet):
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    words = re.sub("(?P<url>https?://[^\s]+)"," ",tweet)
    words = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",words)
    words = REPLACE_NO_SPACE.sub("", words.lower())
    words = REPLACE_WITH_SPACE.sub(" ", words)
    words = str(words)
    return words

In [4]:
test_tweet = tweet_to_words(tweet)
print(test_tweet)

 why not  mocked obama for being black      brexit


Now we need to transformed this cleaned tweet into a bag of words. For this, we need a vocabulary to process it. We need to use the same vocabulary created by the CountVectorizer in the data preparation step. This one we saved as an object in the vocabulary.json file.

In [5]:
import json

version = "combined"
with open(f"data/vocabulary_{version}.json", "r") as file:
    vocabulary = json.load(file)

We will define a function for which we will pass the processed tweet and the vocabulary as parameters and will return a list.
This list can be understood as a row of a dataframe. The list will have the length of the vocabulary (5000) meaning that each element (column) would represent a letter in the vocabulary, and the number in that element will be the count of the word in the new tweet.

In [6]:
def bow_encoding(tweet, vocabulary): #function bag of words encoding.
    bow = [0] * len(vocabulary) # Start by setting the count for each word in the vocabulary to zero. #That is, a list filled 5000 elements, all of them are 0.
    for word in tweet.split():  # For each word in the string
        if word in vocabulary:  # If the word is one that occurs in the vocabulary, increase its count.
            bow[vocabulary[word]] += 1 #the element in the position of the word in the vocabulary will increase by 1 everytime that that words appears in the vocabulary/
    return bow

In [7]:
test_bow = bow_encoding(test_tweet,vocabulary)
print(test_bow)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [8]:
len(test_bow)

5000

## Deployment

For the deployment, first we re create the estimator fit in the last part of the modeling step. 
To do this, we can create a Sagemaker Estimator but using the "attach" method we can link the new estimator to the one created before. We only need to pass the name of the training job used.
This name is the one we saved in the last line in the model notebook.

In [9]:
import json
with open("data/training_job_AWS.json","r") as file:
    training_job = json.load(file)
    training_job = training_job["training_job"]

print(training_job)

xgboost-2022-04-16-12-09-31-972


Now, having the training job, we can associate the new estimator to the training job saved before. That is to say, we re create the estimator.

In [10]:
xgb_combined = sagemaker.estimator.Estimator.attach(training_job)


2022-04-16 12:28:31 Starting - Preparing the instances for training
2022-04-16 12:28:31 Downloading - Downloading input data
2022-04-16 12:28:31 Training - Training image download completed. Training in progress.
2022-04-16 12:28:31 Uploading - Uploading generated training model
2022-04-16 12:28:31 Completed - Training job completed


The next line deploys the model fit before. By deploying it, the function returns the direction of the endpoint where the service is working.

In [12]:
xgb_combined_predictor = xgb_combined.deploy(initial_instance_count = 1, instance_type = 'ml.m4.xlarge',endpoint_name = "twitter-sentiment-endpoint") #we set our own endpoint name

-------!

Now, a library is needed to interact with Amazon services (now and for the lambda function that will be needed later). This library allows to communicate with AWS and can gives us access to the SageMaker environment. We need this access to send and get information from the endpoint deployed. 
The library is boto3

In [13]:
import boto3
runtime = boto3.Session().client('sagemaker-runtime')

The runtime variable is containing the sagemaker runtime. We will use a method from this runtime, this method will allow to get a response from the model created. 
But the name of the endpoint is needed. The name of the endpoint (xgb_combined_predictor) can be accessed by a method:

In [14]:
print(xgb_combined_predictor.endpoint_name)

twitter-sentiment-endpoint


Now a response variable is created by calling (invoking) the endpoint.

In [15]:
response = runtime.invoke_endpoint(EndpointName = xgb_combined_predictor.endpoint_name, # The name of the endpoint we created
                                       ContentType = 'text/csv',                     # The data format that is expected
                                       Body = ','.join([str(val) for val in test_bow]).encode('utf-8'))
#the test is transformed from an integer array to a string document encoded so it can be read by the predictor (endpoint)

In [16]:
print(response)

{'ResponseMetadata': {'RequestId': '936c66b9-f2d9-4888-9589-0bcf628fd178', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '936c66b9-f2d9-4888-9589-0bcf628fd178', 'x-amzn-invoked-production-variant': 'AllTraffic', 'date': 'Sat, 16 Apr 2022 12:59:14 GMT', 'content-type': 'text/csv; charset=utf-8', 'content-length': '18'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv; charset=utf-8', 'InvokedProductionVariant': 'AllTraffic', 'Body': <botocore.response.StreamingBody object at 0x7f43d04ef518>}


The response gotten from the endpoint is in a json format. For this, we nead to read it and decode it. 

In [17]:
response = response['Body'].read().decode('utf-8')

In [18]:
print(response)

0.9954820871353149


In [19]:
print(round(float(response)))

1


We can see that the response 1 (closer to 1), meaning that the model succesfully calsified the example.

Now that we have tested that the endpoint is running, we can set up the rest of the app and for now.

## The endpoint will be deleted when not used, otherwise charges might apply.

In [20]:
xgb_combined_predictor.delete_endpoint()

# Deployment of Web App

For the deployment of the web App two AWS services will be used. The Sagemaker endpoint is not opened to the internet. We should create an interface that can get data from the open internet and communicate with the Sagemaker model. We can create a public API that will recieve the data, then we need a lambda function to transform the data and communicate with the model.

1. API Gateway. This service allows us to create a public URL to where we can send information via a HTTP POST request. The information can be recieved from the opened internet. We can set the API to trigger a Lambda function. That is, to pass the information recieved to the lambda (python) function. The Lambda function will give back the answer from the sagemaker Model and then the API will show the answer.

2. LambdaService. This service is a simple python script. It will save the basic functions that we define to process the data. When the data is processed (to pass from a text to a numeric array of 5000 columns) it will send the information to the Sagemaker endpoint via the boto3 module (getting the Sagemaker runtime). The lambda will recieve the response from the Sagemaker endpoint and it will return this information to the API created. 

### Creation of the Lambda

For the lambda. 
First. We have to create a new Rol in Amazon AIM console. This new Role has to be associated with the AWS Services and linked ot Lambda. The permission must be SageMakerFUllAccess.
Then we need to create the lambda in the Amazon Lambda Console. For this, we choose to create a function and we select th "Author from scratch" option, which lets us set our own code. We select Python as the runtime and then select the Role previously created.
In the Lambda handler part of the function created we should copy the code below, which is a compilation of the test we had done previously. 

In [ ]:
import boto3

# And we need the regular expression library to do some of the data processing
import re

def tweet_to_words(tweet):
    REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
    REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    words = re.sub("(?P<url>https?://[^\s]+)"," ",tweet)
    words = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",words)
    words = REPLACE_NO_SPACE.sub("", words.lower())
    words = REPLACE_WITH_SPACE.sub(" ", words)
    words = str(words)
    return words

def bow_encoding(tweet, vocabulary): #function bag of words encoding.
    bow = [0] * len(vocabulary) # Start by setting the count for each word in the vocabulary to zero. #That is, a list filled 5000 elements, all of them are 0.
    for word in tweet.split():  # For each word in the string
        if word in vocabulary:  # If the word is one that occurs in the vocabulary, increase its count.
            bow[vocabulary[word]] += 1 #the element in the position of the word in the vocabulary will increase by 1 everytime that that words appears in the vocabulary/
    return bow


def lambda_handler(event, context):

    vocab = {'just': 2408, 'finished': 1681, 'with': 4872, 'awesome': 393, 'amp': 237, 'super': 4263, 'staff': 4160, 'im': 2195, 'to': 4461, 'work': 4903, 'together': 4465, 'la': 2477, 'angry': 253, 'birds': 528, 'film': 1668, 'download': 1336, '2016': 30, 'my': 2933, 'new': 2999, 'book': 602, 'cover': 1049, 'so': 4076, 'much': 2917, 'books': 604, 'romance': 3775, 'preorder': 3417, 'love': 2646, 'twitch': 4574, 'instead': 2280, 'in': 2216, 'but': 695, 'international': 2292, 'debate': 1163, 'on': 3131, 'the': 4390, 'best': 488, 'kind': 2440, 'of': 3102, 'happy': 1970, 'friday': 1757, 'yall': 4950, 'mood': 2878, 'funny': 1786, 'instadaily': 2274, 'instamood': 2278, 'weekend': 4788, 'training': 4503, 'fitfam': 1688, 'fitness': 1689, 'tgif': 4376, 'cant': 742, 'wait': 4733, 'go': 1855, 'see': 3887, 'finding': 1675, 'shot': 3973, 'this': 4420, 'months': 2877, 'issue': 2335, 'almost': 207, 'its': 2341, 'quite': 3550, 'plain': 3319, 'and': 248, 'simple': 4004, 'it': 2339, 'works': 4909, 'me': 2754, 'guess': 1935, 'might': 2814, 'be': 445, 'first': 1684, 'very': 4683, 'amen': 226, 'truth': 4553, 'always': 217, 'lt3': 2663, 'blessed': 560, 'theyll': 4411, 'never': 2996, 'understand': 4608, 'boy': 620, 'korean': 2469, 'need': 2974, 'hug': 2136, 'emotional': 1431, 'thank': 4380, 'god': 1859, 'for': 1721, 'because': 457, 'without': 4874, 'city': 865, 'is': 2318, 'right': 3752, 'down': 1335, 'there': 4403, 'cleveland': 882, 'disgraced': 1286, 'someones': 4099, 'excited': 1529, 'about': 93, 'outside': 3188, 'caturday': 771, 'morning': 2888, 'fathers': 1619, 'day': 1148, 'beautiful': 455, 'dads': 1128, 'rest': 3718, 'daddy': 1126, 'til': 4450, 'were': 4802, 'again': 167, 'you': 4970, 'miss': 2847, 'lots': 2643, 'stuff': 4229, 'booked': 603, 'our': 3181, 'open': 3142, 'few': 1660, 'more': 2881, 'details': 1241, 'whole': 4843, 'family': 1598, 'course': 1047, 'one': 3133, 'time': 4452, 'that': 4385, 'all': 196, 'separate': 3908, 'even': 1504, 'turn': 4562, 'light': 2583, 'read': 3609, 'visit': 4708, 'today': 4462, 'fabulous': 1574, 'place': 3317, 'eat': 1390, 'drink': 1349, 'pizza': 3315, 'was': 4760, 'early': 1386, 'gift': 1835, 'got': 1880, 'both': 614, 'sons': 4105, 'feeling': 1639, 'pueorico': 3516, 'vacation': 4661, 'thankful': 4381, 'fo': 1705, 'arkansas': 321, 'strawberries': 4211, 'aint': 183, 'good': 1868, 'superhero': 4264, 'dubai': 1361, 'fathersday': 1620, 'dad': 1125, 'fat': 1616, 'vegas': 4672, 'life': 2577, 'pool': 3366, 'lasvegas': 2494, 'hotel': 2120, 'golden': 1863, 'hope': 2108, 'have': 1999, 'wonderful': 4892, 'observation': 3092, 'friends': 1760, 'fun': 1780, 'summer': 4252, 'holiday': 2092, 'travel': 4508, 'adventure': 147, 'beauty': 456, 'self': 3895, 'confidence': 978, 'face': 1575, 'joytrain': 2400, 'finally': 1673, 'nbafinals': 2965, 'game': 1791, 'an': 242, 'hour': 2122, 'away': 392, 'af': 150, 'cmon': 902, 'warriors': 4757, 'lets': 2552, 'cavs': 779, 'ep': 1474, 'we': 4772, 'discuss': 1280, 'drinking': 1351, 'wants': 4752, 'udtapunjab': 4589, 'online': 3136, 'revenge': 3735, 'comments': 936, 'man': 2707, 'girl': 1840, 'will': 4854, 'ever': 1508, 'her': 2039, 'father': 1617, 'figure': 1666, 'ibiza': 2161, 'calling': 715, 'back': 401, 'home': 2097, 'island': 2330, 'retweeted': 3731, 'amsterdam': 240, 'tomorrow': 4469, 'why': 4848, 'are': 314, 'taking': 4309, 'money': 2870, 'out': 3184, 'account': 111, 'last': 2493, 'did': 1253, 'didnt': 1255, 'previous': 3431, 'tweet': 4567, 'leavers': 2532, 'now': 3066, 'racist': 3570, 'according': 110, 'had': 1950, 'enough': 1466, 'going': 1861, 'bed': 461, 'world': 4911, 'ready': 3612, 'foodporn': 1716, 'help': 2033, 'human': 2141, 'themselves': 4398, 'whats': 4814, 'left': 2543, 'humanity': 2142, 'depressed': 1220, 'hopeful': 2109, 'peace': 3252, 'looking': 2628, 'little': 2602, 'happiness': 1969, 'come': 929, 'join': 2384, 'sad': 3812, 'hear': 2018, 'keep': 2418, 'singing': 4012, 'lady': 2481, 'racism': 3568, 'still': 4189, 'alive': 195, 'latest': 2500, 'thanks': 4382, 'follow': 1709, 'these': 4407, 'your': 4974, 'healthy': 2017, 'stress': 4217, 'free': 1749, 'okay': 3125, 'has': 1983, 'been': 463, 'cannot': 740, 'what': 4810, 'ed': 1399, 'get': 1828, 'up': 4633, 'too': 4473, 'big': 508, 'thankyou': 4384, 'keeping': 2419, 'safe': 3815, 'ill': 2191, 'decide': 1166, 'power': 3397, 'bihday': 517, 'gaming': 1795, 'news': 3002, 'pretty': 3427, 'feel': 1637, 'better': 495, 'tho': 4422, 'tired': 4457, 'e32016': 1381, 'when': 4815, 'order': 3164, 'food': 1714, 'half': 1954, 'ago': 173, 'they': 4409, 'havent': 2000, 'delivered': 1195, 'nature': 2959, 'photography': 3293, 'cards': 751, 'at': 357, 'useful': 4645, 'idiots': 2175, 'being': 475, 'by': 701, 'into': 2298, 'giving': 1847, 'second': 3881, 'amendment': 227, 'update': 4634, 'social': 4079, 'analytics': 245, 'motivation': 2897, 'do': 1308, 'well': 4796, 'yesterday': 4961, 'theres': 4404, 'wednesday': 4785, 'euro2016': 1498, 'hardcore': 1976, 'sexy': 3934, 'teens': 4352, 'want': 4750, 'someone': 4098, 'bit': 531, 'seeing': 3888, 'absolute': 97, 'joy': 2399, 'instalike': 2277, 'instacool': 2273, 'vsco': 4726, 'vscocam': 4727, 'waste': 4763, 'stuck': 4223, 'went': 4801, 'show': 3980, 'shitfest': 3959, 'seriously': 3914, 'conce': 965, 'worst': 4917, 'ive': 2347, 'hes': 2047, 'his': 2072, 'mind': 2825, 'lip': 2596, 'sick': 3990, 'worried': 4914, 'cold': 913, 'things': 4415, 'here': 2042, 'from': 1765, 'beginning': 468, 'fundraising': 1784, 'really': 3620, 'forward': 1733, 'tonight': 4470, 'raising': 3583, 'look': 2625, 'faces': 1580, 'mean': 2756, 'business': 692, 'week': 4787, 'happening': 1965, 'late': 2495, 'night': 3022, 'sessions': 3921, 'pay': 3248, 'orlando': 3173, 'disappointed': 1272, 'true': 4536, 'pain': 3210, 'inside': 2268, 'train': 4502, 'watch': 4765, 'gorilla': 1879, 'simulator': 4008, 'adapt': 133, 'environment': 1472, 'tear': 4343, 'thought': 4427, 'others': 3177, 'who': 4841, 'close': 894, 'them': 4396, 'health': 2016, 'cancer': 733, 'offers': 3110, 'response': 3717, 'shooting': 3969, 'lack': 2479, 'action': 119, 'gun': 1939, 'laws': 2513, 'hillary': 2061, 'isnt': 2331, 'or': 3160, 'shes': 3952, 'anybody': 286, 'consumer': 1000, 'price': 3434, 'index': 2231, 'yoy': 4982, 'may': 2746, 'blog': 569, 'silver': 4002, 'gold': 1862, 'forex': 1728, 'am': 218, 'coffee': 911, 'positive': 3381, 'en': 1440, 'route': 3794, 'conference': 976, 'days': 1150, 'service': 3918, 'dnc': 1307, 'let': 2551, 'say': 3843, 'than': 4379, 'us': 4641, 'most': 2891, 'acting': 118, 'career': 753, 'then': 4399, 'evening': 1505, 'dear': 1158, 'road': 3766, 'many': 2714, 'times': 4453, 'retweet': 3730, 'if': 2180, 'youre': 4975, 'commercial': 938, 'two': 4578, 'dogs': 1314, 'doing': 1316, 'makes': 2692, 'nothing': 3057, 'else': 1422, 'matters': 2743, 'park': 3229, 'people': 3264, 'can': 729, 'lighttherapy': 2585, 'depression': 1222, 'altwaystoheal': 216, 'own': 3199, 'washington': 4761, 'post': 3385, 'talk': 4314, 'shows': 3985, 'neutral': 2994, 'bitch': 532, 'not': 3054, 'nigeria': 3017, 'trying': 4555, 'son': 4102, 'location': 2614, 'play': 3329, 'role': 3773, 'germany': 1826, 'performance': 3270, 'th': 4377, 'babe': 397, 'how': 2127, 'could': 1035, 'lose': 2633, 'faith': 1588, 'relationship': 3661, 'twitter': 4576, 'friendship': 1761, 'cruel': 1098, 'impoant': 2211, 'celebrate': 786, 'anne': 259, 'wedding': 4783, 'nervous': 2988, 'whos': 4846, 'fridayfeeling': 1758, 'relax': 3664, 'oh': 3119, 'done': 1324, 'until': 4632, 'next': 3010, 'tuesday': 4558, 'row': 3796, 'talks': 4316, 'live': 2605, 'welcome': 4795, 'fascism': 1609, 'only': 3137, 'worse': 4916, 'dont': 1326, 'fight': 1663, 'hatred': 1997, 'rip': 3757, 'victims': 4690, 'dick': 1250, 'he': 2004, 'gone': 1866, 'bad': 405, 'dog': 1313, 'country': 1043, 'police': 3352, 'arent': 316, 'any': 285, 'know': 2463, 'changed': 807, 'proud': 3502, 'year': 4955, 'getting': 1830, 'line': 2593, 'smiling': 4062, 'feelings': 1640, 'as': 333, 'often': 3118, 'those': 4424, 'meet': 2769, 'smile': 4058, 'hey': 2048, 'guys': 1943, 'she': 3947, 'number': 3074, 'everywhere': 1517, 'confident': 979, 'alone': 209, 'pic': 3297, 'government': 1884, 'though': 4425, 'monday': 2868, 'mondaymotivation': 2869, 'maybe': 2747, 'take': 4306, 'project': 3475, 'level': 2557, 'woke': 4879, 'before': 465, 'alarm': 189, 'boyfriend': 625, 'ask': 340, 'wouldnt': 4919, 'trade': 4497, 'anything': 289, 'app': 299, 'telling': 4355, 'doesnt': 1312, 'anymore': 287, 'check': 821, 'kids': 2434, 'hosts': 2118, 'steal': 4184, 'dance': 1132, 'music': 2925, 'inspired': 2272, 'needed': 2976, 'waiting': 4735, 'story': 4206, 'like': 2586, 'ramadhan': 3586, 'south': 4118, 'killed': 2436, 'sweet': 4289, 'na': 2939, 'porn': 3375, 'nude': 3073, 'xxx': 4948, 'nasty': 2950, 'young': 4973, 'horny': 2113, 'slut': 4049, 'hot': 2119, 'shy': 3988, 'naughty': 2960, 'kinky': 2450, 'old': 3126, 'guy': 1942, 'off': 3104, 'some': 4095, 'sleep': 4040, 'scotland': 3862, 'needs': 2977, 'dead': 1153, 'bc': 443, 'school': 3854, 'rn': 3764, 'around': 323, 'actually': 129, 'long': 2623, 'calm': 717, 'make': 2689, 'return': 3726, 'trip': 4530, 'village': 4699, 'looks': 2629, 'treat': 4516, 'way': 4770, 'treated': 4517, 'lifestyle': 2582, 'kevin': 2425, 'sat': 3833, 'copy': 1022, 'test': 4372, 'asian': 337, 'another': 266, 'america': 228, 'prayfororlando': 3410, 'happened': 1964, 'couple': 1044, 'launch': 2507, 'team': 4337, 'comes': 931, 'received': 3624, 'card': 750, 'xmas': 4945, 'store': 4204, 'wont': 4893, 'accept': 105, 'says': 3845, 'must': 2929, 'used': 4644, 'color': 922, 'hair': 1953, 'should': 3975, 'possible': 3384, 'omg': 3129, 'available': 386, 'shop': 3971, 'hi': 2049, 'message': 2794, 'no': 3026, 'words': 4902, 'respect': 3714, 'found': 1735, 'delete': 1191, 'tweets': 4571, 'find': 1674, 'deletetweets': 1192, 'sunshine': 4262, 'headed': 2009, 'beach': 446, 'shall': 3940, 'myself': 2936, 'sands': 3829, 'write': 4924, 'epic': 1475, 'under': 4605, 'sunset': 4261, 'babygirl': 400, 'baby': 399, 'cantwait': 743, 'slow': 4045, 'cooking': 1017, 'oil': 3121, 'praying': 3411, 'also': 212, 'sunday': 4256, 'anime': 258, 'clothes': 897, 'try': 4554, 'ideas': 2169, 'list': 2597, 'their': 4395, 'practice': 3402, 'become': 458, 'came': 721, 'fast': 1615, 'cute': 1120, 'same': 3827, 'amazing': 222, 'believe': 478, 'bacon': 404, 'writing': 4926, 'posts': 3390, 'wtf': 4931, 'wrong': 4928, 'florida': 1700, 'warcraft': 4754, 'ugh': 4591, 'officially': 3116, 'launching': 2509, 'afternoon': 165, 'lovely': 2652, 'made': 2677, 'celebration': 789, 'cheers': 825, 'knew': 2459, 'research': 3704, 'everyone': 1513, 'wisdom': 4868, 'reality': 3616, 'clarity': 873, 'gratitude': 1901, 'wednesdaywisdom': 4786, 'takes': 4308, 'special': 4132, 'snapshot': 4071, 'wet': 4807, 'working': 4906, 'playing': 3333, 'melancholy': 2773, 'melancholymusic': 2774, 'stas': 4174, 'minutes': 2839, 'gonna': 1867, 'soon': 4106, 'bought': 616, 'oops': 3140, 'mistake': 2850, 'girlfriend': 1842, 'broke': 659, 'date': 1141, 'centre': 795, 'care': 752, 'black': 539, 'history': 2077, 'month': 2876, 'praise': 3403, 'sets': 3923, 'female': 1649, 'perfect': 3268, 'boys': 626, 'worked': 4904, 'over': 3190, 'years': 4956, 'tears': 4344, 'blue': 578, 'woman': 4880, 'yet': 4963, 'electronic': 1420, 'bogota': 589, 'colombia': 920, 'puntohost': 3519, 'edm': 1403, 'dj': 1304, 'fashion': 1614, 'person': 3278, 'sometimes': 4101, 'simply': 4006, 'means': 2758, 'strong': 4220, 'problems': 3454, 'key': 2426, 'energy': 1456, 'quote': 3552, 'quotes': 3555, 'lessons': 2550, 'hopefully': 2110, 'moving': 2907, 'house': 2124, 'gt': 1925, 'anti': 269, 'ur': 4640, 'shame': 3941, 'bull': 679, 'dominate': 1319, 'direct': 1268, 'whatever': 4813, 'yo': 4966, 'yeah': 4954, 'group': 1919, 'dreams': 1345, 'forgot': 1730, 'lonely': 2622, 'great': 1903, 'set': 3922, 'job': 2377, 'bliss': 564, 'lucky': 2665, 'agreed': 175, 'swimming': 4293, 'does': 1311, 'august': 377, 'shoot': 3967, 'university': 4621, 'students': 4225, 'protesting': 3500, 'against': 168, 'pm': 3341, 'jobs': 2378, 'jobsearch': 2379, 'protests': 3501, 'spread': 4150, 'rain': 3579, 'sadly': 3813, 'later': 2498, 'cat': 765, 'spirit': 4141, 'give': 1844, 'spirituality': 4143, 'fave': 1624, 'pics': 3300, 'grateful': 1900, 'season': 3877, 'info': 2248, 'saturday': 3837, 'yummy': 4987, 'pig': 3305, 'ice': 2163, 'cream': 1072, 'finger': 1679, 'where': 4816, 'mexico': 2801, 'club': 901, 'drinks': 1352, 'yes': 4960, 'relaxed': 3665, 'general': 1813, 'knowledge': 2465, 'successful': 4241, 'euros': 1502, 'while': 4819, 'sounds': 4115, 'youve': 4981, 'seen': 3891, 'through': 4437, 'plan': 3320, 'sun': 4254, 'announce': 262, 'designed': 1229, 'website': 4781, 'which': 4818, 'anniversary': 261, '10': 1, 'adam': 131, 'movies': 2906, 'couldnt': 1037, 'able': 90, 'win': 4858, 'easy': 1389, 'match': 2735, 'lost': 2640, 'form': 1731, 'growing': 1922, 'either': 1415, 'september': 3910, 'congratulations': 987, 'humans': 2144, 'suicide': 4247, 'mass': 2731, 'shootings': 3970, 'etc': 1488, 'moments': 2865, 'flower': 1701, 'lgbt': 2561, 'less': 2548, 'skin': 4027, 'hell': 2030, 'would': 4918, 'anyone': 288, 'prove': 3504, 'secret': 3883, 'having': 2002, 'knowing': 2464, 'already': 211, 'child': 831, 'attacked': 363, 'alligator': 200, 'disney': 1291, 'please': 3335, 'idiot': 2173, 'trump': 4539, 'studio': 4226, 'attack': 362, '3d': 56, 'think': 4416, 'head': 2008, 'empty': 1439, 'each': 1383, 'side': 3992, 'entire': 1468, 'fucked': 1771, 'appreciate': 304, 'flowers': 1702, 'far': 1606, 'eye': 1567, 'lot': 2642, 'todays': 4463, 'street': 4215, 'united': 4618, 'kingdom': 2448, 'production': 3460, 'mom': 2863, 'registered': 3651, 'above': 94, 'expectations': 1545, 'april': 308, 'happiest': 1968, 'known': 2466, 'smiles': 4059, 'niece': 3014, 'via': 4685, 'sign': 3995, 'coming': 933, 'wine': 4860, 'fresh': 1756, 'sea': 3872, 'instagood': 2275, 'forever': 1727, 'loved': 2647, 'movie': 2904, 'screening': 3866, 'ebook': 1393, 'marketing': 2724, 'photooftheday': 3294, 'venue': 4678, 'husband': 2152, 'break': 635, 'ate': 358, 'weight': 4791, 'loss': 2638, 'green': 1908, 'picoftheday': 3299, 'girls': 1843, 'mccain': 2750, 'answer': 268, 'question': 3544, 'completely': 961, 'sell': 3898, 'integrity': 2284, 'july': 2405, 'body': 588, 'boston': 612, 'full': 1778, 'front': 1766, 'page': 3208, 'three': 4435, 'stop': 4200, 'media': 2763, 'red': 3633, 'success': 4240, 'thrive': 4436, 'bitches': 533, 'sleeping': 4041, 'ig': 2182, 'shit': 3958, 'instagram': 2276, 'like4like': 2587, 'fb': 1627, 'paper': 3223, 'town': 4492, 'laugh': 2503, 'enteainment': 1467, 'nice': 3011, 'talking': 4315, 'points': 3347, 'repoed': 3689, 'bus': 689, 'driver': 1354, 'after': 164, 'argument': 318, 'gaza': 1804, 'bible': 505, 'brave': 633, 'herbal': 2041, 'organic': 3168, 'vegan': 4671, 'male': 2697, 'members': 2778, 'quit': 3549, 'whitegenocide': 4826, 'tough': 4486, 'lol': 2619, 'jokes': 2388, 'fatherhood': 1618, 'cud': 1105, 'glad': 1848, 'share': 3943, 'dwyane': 1375, 'wade': 4729, 'die': 1256, 'goodluck': 1871, 'fan': 1602, 'brighton': 649, 'stomping': 4196, 'ground': 1918, 'dinner': 1266, 'running': 3804, 'zero': 4992, 'followers': 1710, 'end': 1444, 'revolution': 3740, 'tony': 4472, 'tip': 4455, 'living': 2608, 'adult': 143, 'tips': 4456, 'event': 1506, 'cut': 1119, 'luck': 2664, 'favorite': 1625, 'princess': 3440, 'class': 874, 'sept': 3909, 'hea': 2005, 'goes': 1860, 'affected': 152, 'act': 117, 'terrorism': 4368, 'yyc': 4990, 'orlandoshooting': 3174, 'watching': 4767, 'english': 1461, 'fans': 1604, 'france': 1743, 'football': 1720, 'hide': 2050, 'hit': 2078, '1000': 3, 'plays': 3334, 'lil': 2592, 'cuz': 1122, 'model': 2859, 'said': 3817, 'customer': 1117, 'garden': 1799, 'homegrown': 2098, 'thing': 4414, 'opening': 3144, 'leather': 2530, 'childrens': 838, 'shoes': 3966, 'texas': 4374, 'reunited': 3732, 'wearing': 4777, 'dress': 1346, '15': 19, 'sale': 3821, 'making': 2694, 'blocked': 568, 'least': 2529, 'hubby': 2134, 'realize': 3618, 'sta': 4156, 'content': 1004, 'types': 4584, 'fired': 1683, 'expect': 1544, '2b': 46, 'boycott': 621, 'holding': 2089, 'weapon': 4774, 'apparently': 301, 'ladies': 2480, 'presidents': 3425, 'obama': 3086, 'realized': 3619, 'internet': 2293, 'troll': 4531, 'multiple': 2920, 'basis': 432, 'pa': 3202, 'beer': 464, 'oitnb': 3123, 'packed': 3204, 'surprise': 4275, 'delivery': 1196, 'gorgeous': 1878, 'selfie': 3897, 'illustration': 2194, 'died': 1257, 'michael': 2807, 'enjoy': 1462, 'gives': 1846, 'therapy': 4402, 'tool': 4475, 'healing': 2015, 'lie': 2574, 'cried': 1085, 'celebrating': 788, 'f4f': 1570, 'lunch': 2669, 'iam': 2160, 'affirmation': 153, 'daughter': 1146, 'toddler': 4464, 'suit': 4249, 'tbt': 4331, 'followme': 1712, 'australian': 382, 'women': 4881, 'pussy': 3529, 'enjoying': 1464, 'loveit': 2649, 'picture': 3301, 'keeps': 2420, 'tell': 4354, 'africa': 160, 'dying': 1376, 'youth': 4978, 'leaving': 2534, 'other': 3176, 'peoples': 3265, 'remember': 3678, 'lead': 2518, 'issues': 2336, 'theyre': 4412, 'afraid': 159, 'abt': 100, 'cannabis': 738, '25': 44, 'till': 4451, 'spain': 4124, 'holidays': 2093, 'rainy': 3581, 'weather': 4779, 'pink': 3308, 'box': 619, 'arrived': 330, 'measured': 2761, 'arms': 322, 'blame': 552, 'customers': 1118, 'cafe': 707, 'imagination': 2197, 'sex': 3925, 'stories': 4205, 'assault': 346, 'low': 2658, 'wish': 4869, 'returns': 3729, 'brings': 653, 'crazy': 1070, 'weeks': 4790, 'cry': 1100, '13': 11, 'scary': 3851, 'systems': 4302, 'thinking': 4417, 'ss': 4154, 'fall': 1592, 'rooster': 3781, 'simulation': 4007, 'climb': 888, 'vast': 4670, 'expanse': 1541, 'mountains': 2898, 'reached': 3605, 'leakage': 2524, 'hello': 2031, 'vote': 4717, 'everything': 1515, 'chicago': 827, 'missing': 2849, 'knows': 2467, 'familytime': 1600, 'thursday': 4442, '360': 55, 'desire': 1230, 'genocide': 1818, 'violence': 4702, 'london': 2621, 'aist': 187, 'podcast': 3343, 'wrote': 4929, 'ahead': 176, 'daily': 1129, 'goal': 1857, 'amwriting': 241, 'car': 749, 'ending': 1447, 'nofilter': 3031, 'dutch': 1373, 'idol': 2176, 'real': 3614, 'problem': 3452, 'lies': 2576, 'within': 4873, 'common': 943, 'matter': 2742, 'injustice': 2256, 'king': 2446, 'guilty': 1937, 'vip': 4704, 'babies': 398, 'dream': 1344, 'natural': 2958, 'bouncingbaby': 617, '50': 63, 'nightclub': 3023, 'happier': 1967, 'blogger': 570, 'tweeted': 4569, 'ukip': 4594, 'generations': 1815, 'crack': 1061, '21': 38, 'photos': 3295, 'loves': 2655, 'bae': 407, 'cars': 763, 'ass': 344, 'everyones': 1514, 'political': 3358, 'rage': 3576, 'typical': 4585, 'indian': 2233, 'politics': 3364, 'aap': 85, 'bjp': 538, 'kid': 2433, 'tragedy': 4500, 'goals': 1858, 'data': 1140, 'gotta': 1881, 'finish': 1680, '12': 8, 'screen': 3865, 'save': 3839, 'run': 3803, 'heabreaking': 2006, 'ripchristinagrimmie': 3759, 'apa': 292, 'prayer': 3408, 'reading': 3610, 'upset': 4638, 'forget': 1729, 'hard': 1975, 'final': 1672, 'battle': 439, 'theme': 4397, 'probably': 3450, 'mindset': 2829, 'leads': 2522, 'cultureofdevelopment': 1110, 'organizations': 3169, 'american': 230, 'emo': 1429, 'likeforlike': 2589, 'shoutout': 3978, 'cross': 1094, 'during': 1372, 'religion': 3670, 'claiming': 871, 'muslim': 2926, 'video': 4693, 'england': 1460, 'beard': 448, 'chase': 818, 'leave': 2531, 'despite': 1234, 'fact': 1582, 'source': 4116, 'him': 2063, 'band': 418, 'song': 4103, 'center': 794, 'unhappy': 4612, 'fear': 1630, 'positivevibes': 3382, 'style': 4234, 'yoga': 4967, 'thursdaythoughts': 4443, 'silly': 4001, 'brother': 662, 'decided': 1167, 'pop': 3369, 'aww': 394, 'bing': 524, 'bong': 596, 'stupidity': 4232, 'put': 3531, 'grade': 1892, 'schools': 3855, 'saw': 3842, 'defense': 1180, 'shout': 3977, 'stay': 4183, 'riding': 3750, 'cycling': 1123, 'major': 2687, 'spos': 4148, 'terror': 4367, 'sadness': 3814, 'thoughts': 4428, 'blonde': 574, 'spend': 4135, 'every': 1510, 'heard': 2019, 'death': 1159, 'adorable': 140, 'product': 3459, 'review': 3738, 'control': 1008, 'moment': 2864, 'rough': 3789, 'clean': 879, 'cousins': 1048, 'sound': 4113, 'snapchat': 4069, 'referendum': 3639, 'draw': 1341, 'such': 4242, 'woh': 4878, 'prayers': 3409, 'something': 4100, 'change': 806, 'lazy': 2515, 'john': 2381, 'miami': 2805, 'restaurant': 3719, 'judge': 2402, 'wouldve': 4920, 'given': 1845, 'called': 713, 'longer': 2624, 'lebron': 2536, 'essentialoils': 1485, 'anxiety': 284, 'begun': 470, 'lifeisgood': 2579, 'relaxing': 3666, 'woohoo': 4899, 'yourself': 4977, 'release': 3667, 'negative': 2978, 'faced': 1577, 'deserve': 1227, 'kicking': 2431, 'weird': 4793, 'continues': 1006, 'strength': 4216, 'repeat': 3686, 'heat': 2022, 'exciting': 1530, 'fell': 1645, 'fire': 1682, 'crew': 1084, 'belgian': 476, 'chocolate': 842, 'cake': 709, 'handmade': 1958, '3rd': 57, 'credit': 1080, 'use': 4643, 'cause': 774, 'offer': 3107, 'pathetic': 3240, 'sorry': 4110, 'teeth': 4353, 'friend': 1759, 'istanbul': 2337, 'children': 837, 'future': 1787, 'india': 2232, 'signed': 3996, 'di': 1247, 'poem': 3344, 'loveislove': 2648, 'breakfast': 636, 'wake': 4737, 'behind': 474, 'company': 950, 'squad': 4153, 'camp': 724, 'buffalo': 676, 'vicinity': 4687, 'homes': 2099, 'pray': 3407, 'mommy': 2866, 'took': 4474, 'smiley': 4060, 'begins': 469, 'hat': 1986, 'design': 1228, 'york': 4969, 'bird': 527, 'families': 1597, 'inspirational': 2271, 'huge': 2137, 'crowd': 1096, 'breast': 639, 'humpday': 2149, 'hungry': 2151, 'classes': 875, 'web': 4780, 'admin': 138, 'cats': 770, 'call': 712, 'journalism': 2394, 'findingdory': 1676, 'brain': 630, '30': 51, 'college': 918, 'affirmations': 154, 'teen': 4350, 'parenting': 3226, 'immature': 2201, 'bestfriend': 489, 'national': 2952, 'lovers': 2654, 'thailand': 4378, 'choose': 847, 'step': 4185, 'once': 3132, 'humor': 2147, 'gifts': 1836, 'goodnight': 1873, 'cool': 1018, 'sexual': 3931, 'refugees': 3644, 'request': 3701, 'christina': 855, 'solve': 4094, 'rose': 3785, 'breathe': 640, 'hand': 1957, 'smoke': 4064, 'walk': 4742, 'air': 184, 'along': 210, 'republicans': 3696, 'obamas': 3089, 'eu': 1493, 'euref': 1495, 'gb': 1805, '20': 26, 'yellow': 4958, 'laughing': 2505, 'ht': 2131, 'soul': 4111, 'broken': 660, 'bored': 608, 'won': 4890, 'socialmedia': 4084, 'newyork': 3009, 'usa': 4642, 'imagine': 2198, 'conversations': 1011, 'episode': 1477, 'co': 904, 'public': 3513, 'hours': 2123, 'happen': 1963, 'congrats': 986, 'thomas': 4423, 'explain': 1552, 'promise': 3479, 'bro': 656, 'expressions': 1558, 'single': 4013, 'shopping': 3972, 'repo': 3688, 'mall': 2704, 'random': 3588, 'bday': 444, 'weekends': 4789, 'shocked': 3964, 'president': 3423, 'oscar': 3175, 'allow': 202, 'listen': 2598, 'nope': 3047, 'french': 1755, 'goodbye': 1869, 'chance': 805, 'differently': 1262, 'infographic': 2249, 'inspiration': 2270, 'hill': 2060, 'reach': 3604, 'target': 4321, 'complete': 960, 'task': 4322, 'survive': 4279, 'listening': 2599, 'anger': 252, 'meditation': 2766, 'tech': 4345, 'bitcoin': 536, 'vs': 4725, 'bunch': 683, 'mongering': 2871, 'xenophobic': 4943, 'jesus': 2365, 'blessings': 561, 'gave': 1801, 'tie': 4449, 'ff': 1661, 'opposite': 3153, 'haha': 1951, 'records': 3631, 'ok': 3124, '2nd': 47, 'bff': 503, 'youd': 4971, 'youll': 4972, 'asking': 342, 'cock': 909, 'growth': 1923, 'balance': 412, 'nutrition': 3079, '247': 42, 'graduated': 1893, 'uk': 4593, '1st': 25, 'cage': 708, 'drawing': 1342, '40': 58, 'wishing': 4871, 'workers': 4905, 'immigrants': 2203, 'walls': 4747, 'russia': 3806, 'sunny': 4260, 'goodtimes': 1874, 'animal': 254, 'blur': 582, 'brought': 664, 'pack': 3203, 'mini': 2832, 'debacle': 1162, 'performing': 3272, 'stage': 4161, 'intellectual': 2285, 'minds': 2828, 'men': 2783, 'inlove': 2257, 'age': 169, 'condition': 971, 'david': 1147, 'star': 4170, 'italy': 2340, 'sold': 4091, 'heights': 2028, 'sleepy': 4043, 'hm': 2083, 'bring': 651, 'heres': 2043, 'hoping': 2112, 'brand': 632, 'boom': 605, 'virgin': 4706, 'employees': 1435, 'culture': 1109, 'guide': 1936, 'june': 2407, 'snap': 4068, 'learn': 2526, 'mindfulness': 2827, 'usual': 4656, 're': 3603, 'properly': 3486, 'round': 3792, 'water': 4768, 'reaction': 3607, 'phone': 3290, 'emotions': 1432, 'frustrated': 1768, 'clinton': 889, 'donald': 1320, 'deal': 1156, 'msg': 2912, 'material': 2738, '72': 72, 'plane': 3321, 'plenty': 3336, 'chat': 819, 'lives': 2606, 'mum': 2921, 'link': 2595, 'inshot': 2267, 'sleeps': 4042, 'dude': 1363, 'wild': 4850, '17': 22, 'busy': 694, 'suppoers': 4269, 'ignorance': 2185, 'indeed': 2230, 'trumplies': 4543, 'trumpuniversity': 4549, 'materi': 2736, 'youtube': 4979, 'channel': 809, '2000': 27, 'yay': 4952, 'damn': 1131, 'paid': 3209, 'married': 2727, 'definition': 1186, 'wanna': 4749, 'buy': 699, 'eggs': 1412, 'land': 2486, 'funday': 1781, 'bigger': 509, 'plans': 3325, 'materia': 2737, 'ukraine': 4595, 'present': 3420, 'dangerous': 1134, 'loveyou': 2656, 'prison': 3444, 'talented': 4313, 'staed': 4159, 'colors': 923, 'sir': 4015, 'fool': 1717, 'liberals': 2570, 'basketball': 434, 'yr': 4984, 'science': 3857, 'using': 4652, 'beat': 451, 'legend': 2547, 'returning': 3727, 'injured': 2255, 'turning': 4564, 'nearly': 2971, 'thats': 4388, 'folks': 1708, 'caused': 775, 'lawofattraction': 2512, 'indians': 2235, 'photo': 3291, 'killing': 2438, 'education': 1406, 'campaign': 725, 'stupid': 4231, 'insane': 2263, 'killer': 2437, 'condolences': 973, 'cuck': 1104, 'album': 190, 'ness': 2989, 'upsetting': 4639, 'hu': 2133, '2017': 34, 'saved': 3840, 'held': 2029, 'stars': 4172, 'shameful': 3942, 'hate': 1987, 'war': 4753, 'shi': 3953, 'pittsburgh': 3313, 'lesson': 2549, 'taught': 4326, 'democracy': 1206, 'shut': 3987, 'cookies': 1016, 'smaller': 4054, 'payday': 3249, 'packing': 3205, 'window': 4859, 'orange': 3162, 'competition': 956, 'name': 2942, 'likes': 2591, 'towards': 4491, 'choices': 844, 'excuse': 1533, 'coach': 905, 'activities': 124, 'exercise': 1536, 'behappy': 471, 'weightloss': 4792, 'feliz': 1644, 'toptags': 4478, 'mad': 2675, 'memory': 2782, 'gay': 1802, 'collection': 916, 'exam': 1524, 'turned': 4563, 'boots': 607, 'ban': 417, 'wall': 4746, 'journalists': 2395, 'biher': 519, 'mexicans': 2800, 'rapists': 3595, 'producer': 3457, 'convinced': 1013, 'staing': 4164, 'wwdc2016': 4935, 'gtgt': 1931, 'tape': 4319, 'text': 4375, 'pass': 3232, 'vine': 4700, 'absolutely': 98, 'childhood': 833, 'breaks': 638, 'cheer': 823, 'puts': 3536, 'seems': 3890, 'beating': 453, 'de': 1152, 'eve': 1503, 'since': 4009, 'tragic': 4501, 'accepting': 108, 'total': 4482, 'liberal': 2568, 'trash': 4507, 'eah': 1384, 'jewelry': 2367, 'burn': 686, 'premiere': 3416, 'office': 3111, 'youtuber': 4980, 'hr': 2128, 'ceo': 796, 'monster': 2873, 'greetings': 1912, 'ones': 3135, 'taken': 4307, 'silence': 3999, 'fought': 1734, 'badly': 406, 'following': 1711, 'loving': 2657, 'relationships': 3662, 'marriage': 2726, 'nose': 3052, 'anyway': 290, 'middle': 2813, 'tour': 4487, 'situation': 4022, 'np': 3069, 'goodmorning': 1872, 'plz': 3340, 'mast': 2734, 'alcohol': 191, 'foodie': 1715, 'wow': 4922, 'johnny': 2382, 'bankrupt': 421, 'sho': 3963, 'parents': 3227, 'sense': 3905, 'filter': 1670, 'swag': 4283, 'viral': 4705, 'naked': 2941, 'guitar': 1938, 'incredible': 2227, 'ride': 3748, 'animals': 256, 'hip': 2067, 'willing': 4855, 'rid': 3746, 'warm': 4755, 'fuck': 1769, 'reason': 3621, 'millennials': 2819, 'excit': 1528, 'click': 884, 'system': 4299, 'minecraft': 2831, 'bear': 447, 'occur': 3097, 'wonder': 4891, 'canada': 730, 'fly': 1703, 'radio': 3575, 'creative': 1076, 'workout': 4907, 'berlin': 485, 'walking': 4744, 'double': 1331, 'glasses': 1852, 'eyes': 1568, 'fucking': 1772, 'anywhere': 291, 'near': 2970, 'note': 3055, 'fine': 1677, 'drivers': 1355, 'dancing': 1133, 'starbucks': 4171, 'wakeup': 4739, 'shine': 3956, 'polar': 3350, 'racing': 3567, 'places': 3318, 'sma': 4052, 'pick': 3298, 'flagday2016': 1693, 'flag': 1692, 'speak': 4129, 'summeime': 4251, 'missed': 2848, 'student': 4224, 'bag': 408, 'mother': 2893, 'ten': 4359, 'feeding': 1636, 'motherfucker': 2894, 'bless': 558, 'asked': 341, 'cup': 1112, 'milk': 2818, 'changes': 808, 'becoming': 460, 'development': 1245, 'retail': 3724, 'events': 1507, 'quiet': 3548, 'jealous': 2360, 'shouldnt': 3976, 'murder': 2923, 'high': 2052, 'profile': 3465, 'le': 2517, 'bob': 586, 'kitty': 2455, 'orangeisthenewblack': 3163, 'season4': 3878, 'released': 3668, 'netflix': 2992, 'raise': 3582, 'singer': 4011, 'touch': 4484, 'believes': 479, 'worldwide': 4913, 'create': 1073, 'spring': 4151, 'winter': 4867, 'leaves': 2533, 'keys': 2427, 'sure': 4273, 'suppo': 4266, 'massive': 2733, 'decision': 1168, 'disgusting': 1289, 'involved': 2308, 'consequences': 993, 'point': 3346, 'promote': 3480, 'destruction': 1239, 'complaint': 959, 'logins': 2617, 'brokers': 661, 'memes': 2780, 'meeting': 2770, 'dr': 1338, 'breaking': 637, 'actor': 125, 'anton': 283, 'dies': 1258, 'grandmother': 1898, 'apps': 307, 'batman': 436, 'hero': 2045, 'dc': 1151, 'novel': 3065, 'immediately': 2202, 'base': 429, 'build': 677, 'fantastic': 1605, 'ratings': 3601, 'except': 1527, 'wales': 4741, 'hypocrite': 2157, 'top': 4476, 'jimmy': 2373, 'bold': 591, 'stone': 4197, 'roses': 3786, 'choice': 843, 'ki': 2428, 'loser': 2634, 'piece': 3304, 'annoyed': 264, 'animation': 257, 'interview': 2296, 'wanted': 4751, 'bs': 669, 'smh': 4057, 'noh': 3033, 'states': 4180, 'fighting': 1665, 'hunger': 2150, 'idwp': 2178, 'shades': 3936, 'kill': 2435, 'guns': 1940, 'sucks': 4245, 'id': 2167, 'begin': 467, 'pride': 3437, 'kinda': 2441, 'scared': 3849, 'senseless': 3906, 'pas': 3231, 'different': 1261, 'mature': 2745, 'gets': 1829, 'civil': 867, 'hands': 1959, 'hopes': 2111, 'prevent': 3429, 'buddy': 675, 'wave': 4769, 'union': 4615, 'robe': 3769, 'network': 2993, 'igers': 2183, 'necklace': 2973, 'nursery': 3077, 'rhymes': 3742, 'lyrics': 2672, 'relevant': 3669, 'advanced': 146, 'tried': 4526, 'false': 1593, 'committed': 942, 'defending': 1179, 'ended': 1445, 'ignored': 2190, 'confused': 985, 'trust': 4552, 'biggest': 510, 'dark': 1137, 'ya': 4949, 'fit': 1687, 'gym': 1946, 'diet': 1259, 'voice': 4712, 'nails': 2940, 'meant': 2759, 'festival': 1657, 'ramadan': 3585, 'muslims': 2928, 'deep': 1173, 'impossible': 2212, 'ways': 4771, 'nobody': 3030, 'joking': 2389, 'poland': 3349, 'comprehend': 962, 'unbelievable': 4602, 'blogging': 572, 'eurgbp': 1496, 'gains': 1789, 'countdown': 1040, 'counting': 1042, 'bestie': 491, 'poor': 3368, 'largest': 2491, 'p2': 3200, 'struggles': 4221, 'learning': 2528, 'among': 236, 'claim': 869, '70': 71, 'e3': 1380, 'terrible': 4366, 'ideology': 2170, 'winner': 4864, 'moved': 2901, 'million': 2820, 'saying': 3844, 'connecting': 991, 'ppl': 3400, 'latepost': 2497, 'remedies': 3677, 'pres': 3419, 'msm': 2913, 'hrc': 2129, 'corrupt': 1029, 'sociopath': 4087, 'repost': 3690, 'memories': 2781, 'surprised': 4276, 'exactly': 1523, 'lousy': 2645, 'council': 1038, 'filled': 1667, 'result': 3722, 'games': 1794, 'joined': 2385, 'planet': 3322, 'created': 1074, 'due': 1365, 'process': 3455, 'driving': 1356, 'angels': 251, 'building': 678, 'silent': 4000, 'poetry': 3345, 'writer': 4925, 'wishes': 4870, 'white': 4823, 'supposed': 4271, 'mins': 2837, 'reasons': 3622, 'wife': 4849, 'advice': 148, 'genuinely': 1821, 'queue': 3546, 'cou': 1033, 'theconjuring2': 4392, 'yours': 4976, 'sohappy': 4090, 'diy': 1303, 'principal': 3441, 'continue': 1005, 'slavery': 4039, 'local': 2613, 'australia': 381, 'net': 2990, 'actual': 128, 'ootd': 3141, 'l4l': 2476, 'roll': 3774, 'graduation': 1895, 'tonights': 4471, 'sales': 3823, 'democratic': 1207, 'nominee': 3039, 'candidate': 735, 'factory': 1584, 'sa': 3811, 'ultra': 4599, 'thx': 4445, 'push': 3526, 'state': 4175, 'politicians': 3363, 'illegal': 2192, 'flights': 1698, 'normal': 3050, 'romantic': 3776, 'detoxdiet': 1242, 'peek': 3257, 'uses': 4647, 'word': 4901, 'mr': 2908, 'beyond': 501, 'goodvibes': 1875, 'mine': 2830, 'room': 3780, 'chose': 849, 'ordered': 3165, 'told': 4466, 'folk': 1707, 'relatives': 3663, 'instapic': 2279, 'rainbow': 3580, 'japan': 2355, 'navy': 2961, 'hospital': 2117, 'bih': 516, 'flying': 1704, 'las': 2492, 'comics': 932, 'delta': 1197, 'duck': 1362, 'law': 2510, 'fl': 1691, 'goodday': 1870, '31': 52, 'horror': 2116, 'freedom': 1750, 'courage': 1046, 'clients': 885, 'abrahamhicks': 95, 'tag': 4305, 'below': 481, 'israel': 2332, 'htt': 2132, 'itself': 2342, 'mirror': 2841, 'overcome': 3191, 'bernie': 487, 'freaking': 1747, 'easily': 1387, 'queen': 3542, 'st': 4155, 'bride': 645, 'monsters': 2874, 'grand': 1897, 'org': 3167, 'workshop': 4910, 'precious': 3413, 'embrace': 1427, 'truly': 4538, 'value': 4663, 'heaven': 2023, 'stand': 4166, 'wear': 4776, 'essential': 1484, 'oils': 3122, 'path': 3239, 'lt': 2662, 'mama': 2706, 'creativity': 1077, 'speaking': 4130, 'neverump': 2998, 'wars': 4758, 'gbp': 1806, 'thousands': 4429, 'search': 3875, 'terrorist': 4369, 'triggered': 4528, 'sky': 4029, 'haters': 1993, 'played': 3330, 'happens': 1966, 'downtown': 1337, 'lovelife': 2651, 'literally': 2601, 'dis': 1270, 'disgraceful': 1287, 'ps4': 3509, 'looki': 2627, 'usually': 4657, 'helping': 2036, 'small': 4053, 'experience': 1548, 'kiss': 2451, 'dive': 1299, 'touches': 4485, 'numbers': 3075, 'divide': 1301, 'polls': 3365, 'rather': 3600, 'study': 4227, 'gop': 1877, 'isis': 2319, 'desperate': 1232, 'gays': 1803, 'donations': 1323, 'global': 1853, 'platform': 3328, 'tv': 4566, 'islam': 2320, 'christian': 852, 'nba': 2964, 'tickets': 4448, 'somehow': 4097, 'mentioned': 2786, 'intolerance': 2299, 'speech': 4134, 'airpo': 186, 'lean': 2525, 'btw': 671, 'laid': 2482, 'funeral': 1785, 'couples': 1045, 'weddingplanning': 4784, 'curse': 1115, 'view': 4696, 'dese': 1226, 'ocean': 3099, '600': 67, '90': 75, 'niggas': 3019, 'met': 2797, 'fail': 1585, 'trending': 4520, 'gif': 1834, '60s': 68, 'violent': 4703, 'nsfw': 3070, 'vendor': 4675, '43': 60, 'letting': 2555, 'patriarchal': 3242, 'misogyny': 2845, '100': 2, 'fake': 1589, 'hasnt': 1985, 'especially': 1483, 'justice': 2409, 'facebook': 1576, 'opi': 3146, 'da': 1124, 'painting': 3211, 'original': 3171, 'rich': 3743, 'race': 3558, 'complain': 957, 'updates': 4635, 'none': 3042, 'theatre': 4391, 'vehicle': 4673, 'nigga': 3018, 'bee': 462, 'unless': 4625, 'hood': 2105, 'attention': 367, 'awards': 391, 'clown': 900, 'plate': 3327, 'bank': 420, '24': 41, 'ireland': 2313, 'recent': 3625, 'feminism': 1650, 'recently': 3626, 'socialjustice': 4082, 'players': 3332, '100k': 5, 'ad': 130, 'worry': 4915, 'xoxo': 4946, 'grace': 1889, 'bridge': 646, 'drive': 1353, 'across': 116, 'policy': 3357, 'dory': 1330, 'hrs': 2130, 'likely': 2590, 'pple': 3401, 'hv': 2153, 'perform': 3269, 'golf': 1864, '4th': 62, 'lover': 2653, 'usgtgtgt': 4648, 'astrologer': 354, 'products': 3462, 'proven': 3506, 'logo': 2618, 'journey': 2396, 'including': 2223, 'moms': 2867, 'giant': 1833, 'poster': 3388, 'bike': 520, 'propey': 3487, 'fab': 1572, 'effects': 1410, 'clock': 893, 'sight': 3994, 'rally': 3584, 'fuher': 1776, 'pakistan': 3212, 'greece': 1906, 'weak': 4773, 'xx': 4947, 'motivated': 2896, 'jo': 2376, 'compassion': 953, 'china': 840, 'zen': 4991, 'recipe': 3627, 'gamer': 1793, 'lawyer': 2514, 'bonding': 595, 'official': 3115, 'bomb': 593, 'euro': 1497, '16': 20, 'photographer': 3292, 'dates': 1142, 'ear': 1385, 'spoke': 4146, 'songs': 4104, 'lifes': 2581, 'purpose': 3524, 'iphone': 2310, 'noticed': 3059, 'sharing': 3944, 'desse': 1235, 'icecream': 2164, 'interested': 2288, 'stoked': 4194, 'periscope': 3275, 'access': 109, 'escape': 1482, 'react': 3606, 'screaming': 3864, 'send': 3902, 'letter': 2554, 'rejected': 3658, 'google': 1876, 'bn': 583, 'makeup': 2693, 'ty': 4581, 'tweeting': 4570, 'bestseller': 492, 'friyay': 1763, 'sit': 4018, 'cheese': 826, 'terry': 4371, 'force': 1723, 'rock': 3770, 'indiedev': 2236, 'gamedev': 1792, 'fed': 1633, 'heading': 2010, 'honestly': 2104, 'defend': 1177, 'average': 387, 'views': 4697, 'fox': 1739, 'fully': 1779, 'move': 2900, 'soft': 4089, 'anal': 243, 'pure': 3523, 'um': 4600, 'wa': 4728, 'amazon': 223, 'kills': 2439, 'himself': 2065, 'ripchristina': 3758, 'sister': 4016, 'bestfriends': 490, 'pops': 3370, 'brexit': 644, 'led': 2538, 'born': 610, 'showed': 3981, 'stunning': 4230, 'libtard': 2571, 'liar': 2564, 'realise': 3615, 'required': 3702, 'bright': 648, 'spot': 4149, 'meme': 2779, 'personalised': 3279, 'bout': 618, 'challenge': 800, 'posted': 3387, 'strategy': 4210, 'kitchen': 2454, 'delicious': 1194, 'winning': 4865, 'actions': 120, 'rights': 3755, 'vibes': 4686, 'quick': 3547, 'minute': 2838, 'between': 496, 'tampa': 4318, 'serious': 3913, 'promoting': 3481, 'wasted': 4764, 'definitely': 1185, 'souls': 4112, 'meds': 2768, 'thinks': 4418, 'clear': 880, 'catch': 766, 'add': 134, 'comment': 934, 'agree': 174, 'quoteoftheday': 3554, 'demand': 1202, 'stick': 4188, 'heas': 2021, 'christinagrimmie': 856, 'nut': 3078, 'throw': 4438, 'fa': 1571, 'passed': 3233, 'exams': 1525, 'difference': 1260, 'punjab': 3518, 'meanwhile': 2760, 'yum': 4986, 'hiphop': 2068, 'shower': 3982, 'dollars': 1318, 'wood': 4896, 'results': 3723, 'charity': 815, 'curriculum': 1114, 'vids': 4695, 'passing': 3234, 'clearly': 881, 'showing': 3983, 'pls': 3339, 'whoa': 4842, 'incident': 2220, 'canadian': 731, 'four': 1737, 'greatest': 1904, '2008': 28, 'moon': 2879, '11': 6, 'pet': 3283, 'west': 4804, 'causing': 777, 'hoodie': 2106, 'rants': 3590, 'ka': 2415, 'jan': 2352, 'whilst': 4820, 'fair': 1587, 'feet': 1642, 'series': 3912, 'losing': 2637, 'americans': 231, 'lied': 2575, 'werent': 4803, 'print': 3442, 'fellow': 1646, 'lake': 2483, 'mc': 2749, 'lo': 2610, 'causes': 776, 'lee': 2540, 'regret': 3654, 'selected': 3894, 'enemies': 1454, 'laughs': 2506, 'player': 3331, 'drug': 1359, 'difficult': 1263, 'fatty': 1621, 'church': 861, 'christ': 851, 'higher': 2053, 'traveling': 4509, 'january': 2354, 'favourite': 1626, 'seconds': 3882, 'market': 2723, 'blm': 566, 'suppoer': 4268, 'obvious': 3096, 'dm': 1306, 'frank': 1745, 'tune': 4559, 'sing': 4010, 'nyc': 3082, 'kick': 2429, 'eng': 1457, 'chick': 828, 'suppoing': 4270, 'brilliant': 650, 'smell': 4056, 'grimmie': 1915, 'chill': 839, 'visiting': 4709, 'idea': 2168, 'beware': 499, 'classroom': 877, 'effo': 1411, 'spark': 4127, 'joie': 2383, 'space': 4123, 'information': 2250, 'voter': 4719, 'leftist': 2544, 'turns': 4565, 'conflict': 984, 'onto': 3139, 'wellness': 4799, '5th': 66, 'gen': 1809, 'ios': 2309, 'past': 3236, 'current': 1113, 'reagan': 3613, 'bonus': 598, 'heal': 2014, 'gin': 1838, 'newcastle': 3000, 'area': 315, 'radical': 3572, 'exist': 1538, 'ends': 1452, 'europe': 1500, 'harder': 1978, 'kindness': 2445, 'cabinet': 705, 'caoons': 745, 'safety': 3816, 'felt': 1648, 'smoking': 4065, 'industry': 2240, 'capitalism': 747, 'logic': 2616, 'calls': 716, 'videos': 4694, 'lines': 2594, 'block': 567, 'taste': 4323, 'hold': 2088, 'toronto': 4480, 'cooper': 1019, 'crying': 1101, 'accepted': 107, 'cutest': 1121, 'wang': 4748, 'employee': 1434, 'inn': 2259, 'cares': 756, 'record': 3629, 'shift': 3954, 'period': 3274, 'holy': 2096, 'ball': 414, 'sent': 3907, 'attempt': 365, 'horrible': 2114, 'roots': 3784, 'disgusted': 1288, 'straight': 4208, 'boat': 585, 'weve': 4808, 'rules': 3802, 'generation': 1814, 'patience': 3241, 'kelly': 2421, 'twice': 4572, 'benefits': 483, 'wed': 4782, 'jihad': 2371, 'press': 3426, 'politician': 3362, 'spin': 4139, 'agenda': 171, 'myth': 2938, 'hawaii': 2003, 'ugly': 4592, 'maria': 2718, 'interesting': 2289, 'nu': 3071, 'reminder': 3679, 'mail': 2684, '18': 23, '95': 79, 'community': 948, 'victim': 4688, 'manchester': 2710, 'dealing': 1157, 'teacher': 4335, 'hearing': 2020, 'innocent': 2261, 'maga': 2681, 'voting': 4722, 'policies': 3355, 'everyday': 1512, 'swim': 4292, 'barry': 428, 'nations': 2956, 'zuma': 4998, 'former': 1732, 'count': 1039, 'criminals': 1089, 'taxpayers': 4330, 'aicle': 179, 'referring': 3640, 'chicks': 829, 'handsome': 1960, 'corruption': 1031, 'becomes': 459, 'case': 764, 'mcdonalds': 2751, 'fries': 1762, 'main': 2686, 'focus': 1706, 'management': 2708, 'mexican': 2799, 'leadership': 2520, 'glass': 1851, 'planning': 3324, 'potter': 3393, 'minions': 2833, 'victory': 4692, 'blowing': 576, 'removed': 3683, 'huh': 2140, 'shocking': 3965, 'bihdaygirl': 518, 'makeamericagreatagain': 2690, 'loa': 2611, 'nyt': 3084, 'manager': 2709, 'clouds': 899, 'apple': 303, 'salad': 3820, 'spo': 4145, 'ex': 1522, 'heabroken': 2007, 'progressives': 3472, 'lord': 2631, 'looked': 2626, 'soundcloud': 4114, 'commenting': 935, 'feed': 1635, 'blood': 575, 'spent': 4136, 'watched': 4766, 'excellent': 1526, 'east': 1388, 'puppy': 3522, 'officers': 3113, 'sins': 4014, 'type': 4583, 'african': 161, 'dumb': 1370, 'educate': 1404, 'aus': 378, 'brown': 665, '35': 54, 'syndrome': 4297, 'seeks': 3889, 'names': 2944, 'learned': 2527, 'fragile': 1742, 'serve': 3916, 'iran': 2312, 'hater': 1992, '123': 9, 'prince': 3439, 'massage': 2732, 'soreloser': 4108, 'tennessee': 4360, 'alabama': 188, 'bringing': 652, 'address': 136, '14': 15, 'icymi': 2166, 'totally': 4483, 'standing': 4167, 'belief': 477, 'groom': 1916, 'toward': 4490, 'irish': 2314, 'stream': 4214, 'ring': 3756, 'virginia': 4707, 'celtic': 792, 'protesters': 3499, 'kicked': 2430, 'program': 3469, 'somebody': 4096, 'hugs': 2139, 'peaceful': 3253, 'third': 4419, 'et': 1487, 'mylove': 2935, 'comedy': 930, 'celebrity': 791, 'genius': 1817, 'governments': 1885, 'risk': 3762, 'nation': 2951, 'allowed': 203, 'bet': 493, '02': 0, 'venezuela': 4676, 'magical': 2683, 'ceain': 784, 'sundayfunday': 4257, 'sm': 4051, 'pulse': 3517, 'announces': 263, 'scene': 3853, 'interest': 2287, 'entrepreneur': 1471, 'pastor': 3237, 'outrageous': 3187, 'wh': 4809, 'notice': 3058, 'nights': 3024, 'ps': 3508, 'immigration': 2204, 'films': 1669, 'eh': 1414, 'tea': 4333, 'everybody': 1511, 'shis': 3957, 'empire': 1433, 'tattoo': 4325, 'sushi': 4281, 'shake': 3939, 'sisters': 4017, 'powerful': 3398, 'madrid': 2679, 'crap': 1065, 'ticket': 4447, 'ambassador': 224, 'christmas': 857, 'connected': 990, 'frnds': 1764, 'brothers': 663, 'election': 1418, 'yep': 4959, 'fooled': 1718, 'experiences': 1550, 'ha': 1947, 'uncle': 4604, 'majority': 2688, 'drop': 1357, 'prank': 3405, 'peh': 3259, 'proudly': 3503, 'volunteer': 4715, 'oppounity': 3155, 'hates': 1994, 'offensive': 3106, 'feelgood': 1638, 'rate': 3598, 'walked': 4743, 'disrespectful': 1294, 'interracial': 2294, 'terrorists': 4370, 'position': 3380, 'lgbti': 2562, 'candidates': 736, 'scum': 3869, 'blackandwhite': 540, 'democrats': 1209, 'duke': 1368, 'edinburgh': 1401, 'fruit': 1767, 'checked': 822, 'indigenous': 2238, 'lure': 2670, 'incredibly': 2228, 'grow': 1921, 'opinion': 3147, 'grandpa': 1899, 'closer': 896, 'progress': 3470, 'witness': 4875, 'lame': 2484, 'juan': 2401, 'reflections': 3641, 'sides': 3993, 'levels': 2558, 'psychological': 3511, 'teaching': 4336, 'exposing': 1557, 'denial': 1213, 'cameron': 722, '14th': 18, 'yur': 4989, 'unforgettable': 4611, 'joke': 2387, 'league': 2523, 'endless': 1448, 'whose': 4847, 'boos': 606, 'truck': 4534, 'censorship': 793, 'govt': 1886, 'represent': 3693, 'protect': 3495, 'paris': 3228, 'transformation': 4505, 'disgrace': 1285, 'pound': 3395, 'signs': 3997, 'basically': 431, 'degree': 1190, 'graduates': 1894, 'board': 584, 'fraud': 1746, 'treats': 4518, 'ignore': 2189, 'pets': 3284, 'bears': 449, 'firstworldproblems': 1686, 'foreign': 1725, 'increased': 2226, 'mi': 2804, 'defeat': 1175, 'enemy': 1455, 'mens': 2784, 'em': 1424, 'cops': 1021, 'shitty': 3962, 'newyearseve': 3008, 'asks': 343, 'fixed': 1690, 'talent': 4312, 'named': 2943, 'arrive': 329, 'eating': 1391, 'angel': 250, 'slutty': 4050, 'tuned': 4560, 'outrage': 3186, 'thread': 4430, 'language': 2488, 'speaks': 4131, 'damage': 1130, 'pipe': 3309, 'leeds': 2541, 'birmingham': 530, 'spiritual': 4142, 'ignorant': 2187, 'european': 1501, 'balls': 415, 'cox': 1058, 'drugs': 1360, 'lying': 2671, 'antiamerican': 270, 'version': 4682, 'gbpusd': 1807, 'fuel': 1774, 'camps': 726, 'sarcasm': 3832, 'seat': 3879, 'roads': 3767, 'blast': 554, 'nick': 3012, 'eac': 1382, 'door': 1329, 'pc': 3251, 'anonymous': 265, 'developer': 1244, 'offering': 3109, 'station': 4181, 'expe': 1543, 'ufc': 4590, 'rapist': 3594, 'wasnt': 4762, 'merica': 2790, 'wins': 4866, 'influence': 2247, 'bbq': 442, 'mic': 2806, 'con': 963, 'session': 3920, 'exposed': 1556, 'glasgow': 1850, 'panties': 3221, 'toy': 4494, 'losangeles': 2632, 'california': 711, 'cow': 1053, 'optimistic': 3159, 'emails': 1425, 'clothing': 898, 'klan': 2458, 'staup': 4182, 'shairi': 3938, 'merely': 2789, 'hired': 2070, 'boss': 611, 'rubio': 3798, 'award': 390, 'absence': 96, 'although': 214, 'economy': 1398, 'perspective': 3281, 'feminist': 1654, 'patriarchy': 3243, 'altright': 215, 'butt': 696, 'putin': 3532, 'opposition': 3154, 'antiblack': 271, 'scores': 3861, 'terms': 4364, 'creating': 1075, 'respond': 3715, 'reveal': 3733, 'bts': 670, 'chair': 799, 'miles': 2816, 'b4': 396, 'cop': 1020, 'marijuana': 2719, 'arrests': 327, 'assholes': 349, 'ref': 3637, 'hole': 2091, 'roof': 3779, 'leading': 2521, 'meal': 2755, 'direction': 1269, 'indiegamedev': 2237, 'preordered': 3418, 'indecent': 2229, 'abusive': 102, 'lecture': 2537, 'bodies': 587, 'vogue': 4711, 'covering': 1051, 'carolina': 762, 'odd': 3100, 'uae': 4588, 'bollywood': 592, 'bush': 690, 'donaldtrump': 1321, 'values': 4665, 'mario': 2721, 'greed': 1907, '911': 76, 'funding': 1783, 'studying': 4228, 'crackers': 1064, 'eternity': 1489, 'advocate': 149, 'gig': 1837, 'challenged': 801, 'inevitable': 2242, 'defeated': 1176, 'german': 1824, 'evil': 1519, 'caoon': 744, 'transition': 4506, 'harassment': 1974, 'workplace': 4908, 'merry': 2792, 'newyear': 3006, 'bye': 702, 'shooter': 3968, 'prospect': 3493, 'extremism': 1564, 'stalking': 4165, 'bully': 681, 'destroy': 1237, 'suck': 4243, 'spam': 4125, 'amanda': 219, 'whites': 4835, 'joint': 2386, 'household': 2125, 'solutions': 4093, 'day1': 1149, 'remains': 3674, 'republican': 3695, 'victoria': 4691, 'existence': 1539, 'pictures': 3303, 'potential': 3392, 'thieves': 4413, 'famous': 1601, 'snow': 4072, 'launched': 2508, 'mystery': 2937, 'evidence': 1518, 'assume': 353, 'befriending': 466, 'instrumental': 2282, 'track': 4496, 'beaten': 452, 'remove': 3682, 'marks': 2725, 'otherwise': 3178, 'tale': 4311, 'nationalanthem': 2954, 'breitba': 643, 'paying': 3250, 'teenager': 4351, 'bread': 634, 'math': 2741, 'millions': 2821, 'non': 3040, 'vintage': 4701, 'conservatives': 995, 'added': 135, 'rise': 3761, 'noone': 3046, 'tolerance': 4467, 'bunny': 684, 'spokesperson': 4147, 'lg': 2560, 'conservative': 994, 'proof': 3483, 'gf': 1831, 'traffic': 4498, 'ads': 142, 'dig': 1264, 'credibility': 1079, 'southafrica': 4119, 'loses': 2636, 'character': 813, 'passion': 3235, 'jim': 2372, 'sic': 3989, 'trumps': 4546, 'menu': 2787, 'acts': 127, 'yea': 4953, 'everytime': 1516, 'productive': 3461, 'sums': 4253, 'burning': 687, 'coverage': 1050, 'magic': 2682, 'highest': 2055, 'failing': 1586, 'charge': 814, 'paul': 3246, 'peruvian': 3282, 'dept': 1223, 'bin': 523, 'hitting': 2081, 'behaving': 472, 'resign': 3707, 'joe': 2380, 'accused': 113, 'explains': 1553, 'homophobia': 2101, 'excuses': 1534, 'calgary': 710, 'soros': 4109, 'feels': 1641, 'cancel': 732, 'melanin': 2775, 'profit': 3467, 'society': 4085, 'rant': 3589, 'fbi': 1628, 'caught': 773, 'scars': 3850, 'disease': 1283, 'describe': 1225, 'parent': 3224, 'challenging': 802, 'effective': 1409, 'trumptrain': 4548, 'crime': 1086, 'britain': 654, 'puppet': 3521, 'sheep': 3948, 'tells': 4356, 'latina': 2501, 'encouraging': 1443, 'styles': 4235, 'bar': 424, 'iq': 2311, 'temple': 4357, 'wings': 4863, 'motivate': 2895, 'soccer': 4078, 'administration': 139, 'billion': 522, 'racists': 3571, 'onelove': 3134, 'padded': 3207, 'diaz': 1249, 'constantly': 998, 'valid': 4662, 'carl': 757, 'ca': 703, 'six': 4023, 'mouthpiece': 2899, 'mark': 2722, 'concept': 966, 'gender': 1810, 'ohio': 3120, 'atlanta': 360, 'presented': 3421, 'cows': 1057, 'microsoft': 2812, 'extremely': 1563, 'mrs': 2911, 'yrs': 4985, 'careful': 754, 'fck': 1629, 'lowest': 2659, 'depressing': 1221, 'flight': 1697, 'offered': 3108, 'ran': 3587, 'hatefear': 1990, 'shepherd': 3951, 'cc': 781, 'chronic': 860, 'bullying': 682, 'hang': 1961, 'intelligence': 2286, 'libtards': 2572, 'chosen': 850, 'stones': 4198, 'fascist': 1611, 'cnn': 903, 'uneducated': 4609, 'code': 910, 'sitting': 4021, 'temptation': 4358, 'per': 3267, 'hilarious': 2059, 'pisses': 3311, 'persons': 3280, 'wo': 4877, 'theyd': 4410, 'rude': 3799, 'wp': 4923, 'mf': 2802, 'amateur': 221, 'potus': 3394, 'christianity': 853, 'bullshit': 680, 'inc': 2218, 'pigs': 3307, 'seal': 3873, 'stages': 4162, 'struggling': 4222, 'giphy': 1839, 'clip': 892, 'impoance': 2210, 'forecasted': 1724, 'rewrite': 3741, 'proves': 3507, 'endhate': 1446, 'trial': 4524, 'ashamed': 335, 'neighborhood': 2981, 'aloha': 208, 'meaningless': 2757, 'neck': 2972, 'voted': 4718, 'site': 4019, 'historic': 2075, 'rule': 3801, 'hashtag': 1984, 'propaganda': 3484, 'crude': 1097, 'duty': 1374, 'sainsburys': 3818, 'minorities': 2834, 'prices': 3435, 'ebay': 1392, 'preach': 3412, 'abuse': 101, 'duh': 1367, 'attacks': 364, 'performer': 3271, 'binge': 525, 'sciencefiction': 3858, 'absurd': 99, 'bruh': 667, 'claims': 872, 'minded': 2826, 'humour': 2148, 'ambition': 225, 'mustread': 2930, 'invited': 2307, 'obesity': 3090, 'survivor': 4280, 'deeply': 1174, 'posing': 3379, 'boo': 600, 'scream': 3863, 'criticism': 1091, 'bitter': 537, 'resolution': 3712, 'expected': 1546, 'dems': 1212, 'christians': 854, 'hypocritical': 2158, 'serving': 3919, 'blind': 563, 'term': 4362, 'unfit': 4610, 'groups': 1920, 'tits': 4458, 'hepburn': 2038, 'huffington': 2135, 'questioned': 3545, 'statements': 4179, 'architecture': 313, 'defined': 1183, 'accent': 104, 'prevail': 3428, 'bf': 502, 'garbage': 1798, 'penis': 3260, 'rotten': 3788, 'kindle': 2442, 'hiding': 2051, 'knocks': 2462, 'vanity': 4668, 'deliberately': 1193, 'bitching': 535, 'whining': 4822, 'statement': 4177, 'alleged': 198, 'russian': 3807, 'exhausted': 1537, 'dare': 1136, 'surrounded': 4278, 'discussing': 1282, 'cave': 778, 'liked': 2588, 'locks': 2615, 'quoted': 3553, 'scifi': 3860, 'sweety': 4290, 'blatantly': 556, 'colleague': 915, 'diversity': 1300, 'rape': 3592, 'condemn': 968, 'equality': 1479, 'lows': 2660, 'southern': 4120, 'champ': 803, 'sparks': 4128, 'fav': 1623, 'mil': 2815, 'member': 2777, 'dollar': 1317, 'cities': 863, 'sites': 4020, 'attitude': 369, 'dating': 1144, 'doctor': 1309, 'ale': 192, 'graffiti': 1896, 'mostly': 2892, 'noise': 3035, 'gangs': 1797, 'impression': 2213, 'remarks': 3676, 'greatness': 1905, 'hardly': 1979, 'woodrowwilson': 4898, 'grp': 1924, 'bill': 521, 'extinction': 1560, 'bloggersblast': 571, 'british': 655, 'nationalist': 2955, 'resistance': 3711, 'politically': 3360, 'schuster': 3856, 'deaths': 1161, 'holds': 2090, 'remain': 3672, 'proved': 3505, 'citizens': 864, 'sexism': 3927, 'raging': 3577, 'machine': 2674, 'colour': 924, 'empower': 1436, 'divisive': 1302, 'bbc': 441, 'chain': 798, 'facility': 1581, 'jew': 2366, 'editorial': 1402, 'asia': 336, 'private': 3445, 'destroyed': 1238, 'scientists': 3859, 'encounters': 1442, 'fred': 1748, 'toured': 4488, 'based': 430, 'asses': 347, 'chops': 848, 'poc': 3342, 'qualified': 3538, 'positivity': 3383, 'islamic': 2323, 'extremists': 1566, 'traitor': 4504, 'leader': 2519, 'solidarity': 4092, 'models': 2860, 'blacks': 548, 'heck': 2025, 'madness': 2678, 'guard': 1933, 'scumbag': 3870, 'confirm': 980, 'blunt': 580, 'prompts': 3482, 'idolized': 2177, 'sucked': 4244, 'topic': 4477, 'deadliest': 1154, 'incite': 2222, 'gutter': 1941, 'stockholm': 4192, 'bud': 672, 'nor': 3049, 'classy': 878, 'loose': 2630, 'japanese': 2356, 'fyi': 1788, 'george': 1822, '21st': 39, 'coz': 1059, 'allowing': 204, 'hacking': 1949, 'hype': 2155, '24hrs': 43, 'dudes': 1364, 'loyal': 2661, 'prez': 3433, 'austin': 380, 'losses': 2639, 'hoe': 2087, 'aims': 182, 'inse': 2265, 'voters': 4721, 'msnbc': 2914, 'bigotry': 514, 'simon': 4003, 'farmer': 1608, 'congress': 988, 'felony': 1647, 'screenshot': 3867, 'spectrum': 4133, 'banks': 422, 'threats': 4434, 'popular': 3371, 'fetish': 1658, 'regarding': 3649, 'idiotic': 2174, 'vital': 4710, 'housewife': 2126, 'sees': 3892, 'congressman': 989, 'breed': 641, 'blamed': 553, 'wildlife': 4852, 'prostitutes': 3494, 'minority': 2835, 'ego': 1413, 'bastard': 435, 'headlines': 2012, 'knitting': 2461, 'movement': 2902, 'narrative': 2948, 'sending': 3903, 'contender': 1003, 'distractions': 1296, 'elected': 1417, 'reg': 3648, 'omfg': 3128, 'stood': 4199, 'march': 2717, 'blowjob': 577, 'arabian': 310, 'reversed': 3737, 'upon': 4637, 'delusional': 1199, 'airlines': 185, 'bailey': 410, 'darn': 1139, 'critics': 1092, 'creepy': 1081, 'environmental': 1473, 'economic': 1396, 'celebrates': 787, 'crisis': 1090, 'ouch': 3180, 'shown': 3984, 'publicly': 3514, 'crashes': 1068, 'slug': 4046, 'researchers': 3705, 'harm': 1981, 'tcot': 4332, 'refer': 3638, 'tax': 4327, 'shouts': 3979, 'asshat': 348, 'inhumane': 2253, 'perks': 3276, 'sam': 3826, 'expose': 1555, 'seattle': 3880, 'reminding': 3680, 'lgbtq': 2563, 'ove': 3189, 'afghanistan': 156, 'emojis': 1430, 'hollywood': 2095, 'repping': 3691, 'actors': 126, 'publishing': 3515, 'piss': 3310, 'caer': 706, 'mixed': 2852, 'ny': 3081, 'heels': 2027, 'teach': 4334, 'hire': 2069, 'craving': 1069, 'marine': 2720, 'impo': 2209, 'montana': 2875, 'turmoil': 4561, 'dem': 1200, 'nicola': 3013, 'impressions': 2214, 'suspected': 4282, 'globe': 1854, 'childlabor': 835, 'criminal': 1088, 'chief': 830, 'rac': 3557, 'odds': 3101, 'controversy': 1010, 'erased': 1480, 'kindly': 2444, 'broader': 658, 'beast': 450, 'confession': 977, 'lib': 2565, 'gov': 1882, 'demanding': 1203, 'rnc': 3765, 'spanish': 4126, 'profiling': 3466, 'murdering': 2924, 'outlets': 3185, 'controlled': 1009, 'written': 4927, 'yup': 4988, 'experienced': 1549, 'unsatisfied': 4630, 'hispanic': 2073, 'accountable': 112, 'explanation': 1554, 'narcissist': 2947, 'afp': 158, 'havin': 2001, 'kenya': 2424, 'ccot': 782, 'arrested': 325, 'considering': 997, 'ally': 206, 'bigot': 512, 'acknowledge': 115, 'racial': 3565, 'seller': 3899, 'blatant': 555, 'owes': 3198, 'uks': 4597, 'tlc': 4459, 'weirdo': 4794, 'cultures': 1111, 'behavior': 473, 'tyranny': 4586, 'mo': 2856, 'unite': 4616, 'protection': 3496, 'headline': 2011, 'equal': 1478, 'mob': 2857, 'sickening': 3991, 'population': 3373, 'apes': 295, 'root': 3782, 'tries': 4527, 'bot': 613, 'oppose': 3151, 'trophy': 4532, 'mental': 2785, 'thrown': 4439, 'brief': 647, 'analysis': 244, 'lights': 2584, 'hateful': 1991, 'grab': 1887, 'creature': 1078, 'historically': 2076, 'oppressed': 3156, 'prevents': 3430, 'miriam': 2840, 'georgia': 1823, 'familys': 1599, 'discussed': 1281, 'ash': 334, 'educational': 1407, 'voices': 4713, 'anchor': 247, 'bots': 615, 'nm': 3025, 'insults': 2283, 'neutrality': 2995, 'consideration': 996, 'parliament': 3230, 'banned': 423, 'disenfranchised': 1284, 'inequality': 2241, 'closed': 895, 'arabic': 311, 'perhaps': 3273, 'celebrities': 790, 'guardian': 1934, 'strawberry': 4212, 'sexest': 3926, 'boricua': 609, 'filth': 1671, 'dismantle': 1290, 'rates': 3599, 'chinese': 841, '13th': 13, 'commit': 941, 'bats': 437, 'officer': 3112, 'teapay': 4342, 'expes': 1551, 'flapping': 1694, 'malcolm': 2696, 'appalling': 300, 'reveals': 3734, 'endorses': 1451, 'wing': 4861, 'apologies': 296, 'pot': 3391, 'threatening': 4433, 'boil': 590, 'imwithher': 2215, '2a': 45, 'facepalm': 1579, 'uniteblue': 4617, 'medicine': 2765, 'commercials': 939, 'tackling': 4303, 'syria': 4298, 'littlemix': 2604, 'yong': 4968, 'cuba': 1103, 'ibooks': 2162, 'aicles': 180, 'decades': 1165, 'pushing': 3527, 'belongs': 480, 'usher': 4649, 'dignity': 1265, 'nigger': 3020, 'zurich': 4999, 'stolen': 4195, 'rwnj': 3810, 'stereotype': 4186, 'slave': 4035, 'justsaying': 2413, 'confirmation': 981, 'ottawa': 3179, 'attorney': 370, 'marx': 2728, 'bald': 413, 'homophobic': 2102, 'chaos': 811, 'ourselves': 3182, 'engagement': 1459, 'pennsylvania': 3262, 'apaheid': 293, 'activists': 123, 'pos': 3378, 'hillaryclinton': 2062, 'elitists': 1421, 'elect': 1416, 'undermine': 4607, 'blackpeople': 546, 'harlem': 1980, 'whitepeople': 4829, 'latinos': 2502, 'donkey': 1325, 'zionazis': 4994, 'palestinian': 3216, 'semitic': 3901, 'putinschoice': 3534, 'stopracism': 4201, 'decolonizing': 1171, 'decolonization': 1170, 'leejasper': 2542, 'news24': 3003, 'carefully': 755, 'reconciliation': 3628, 'vengeance': 4677, 'blacklivesmatter': 543, 'allows': 205, 'bigots': 515, 'racebaiter': 3559, 'slaveowner': 4037, 'acceptable': 106, 'milo': 2822, 'yiannopoulosis': 4965, 'nazi': 2962, 'tossing': 4481, 'whores': 4845, 'amandanunes': 220, '200k': 29, 'rondarousey': 3778, 'sophisticated': 4107, 'coulda': 1036, 'blicqer': 562, 'morgan': 2883, 'freeman': 1753, 'responded': 3716, 'morganfreeman': 2884, 'commonsense': 944, 'sjw': 4024, 'buttholemouth': 697, 'whiteprivelage': 4832, 'daddygavemeamillion': 1127, 'allahsoil': 197, 'substantially': 4237, 'larger': 2490, 'uskkk': 4653, 'crazybengiefbps': 1071, 'progressive': 3471, 'reactionary': 3608, 'thoughj': 4426, 'caucasians': 772, 'cowboys': 1055, 'newswithed': 3005, 'israeli': 2333, 'settlements': 3924, 'slammed': 4032, 'treacherous': 4512, 'antisemite': 278, 'arabs': 312, 'preying': 3432, 'slavelabor': 4036, 'arrogant': 331, 'discriminating': 1278, 'whitesupremacy': 4838, 'fakenews': 1590, 'paladino': 3213, 'kkk': 2456, 'abetting': 89, 'dylann': 1377, 'competency': 955, 'ignorancehate': 2186, 'neighbors': 2983, 'hatredgreedamp': 1998, 'livesbut': 2607, 'newyear2017': 3007, 'relates': 3659, 'lifelessons': 2580, 'grew': 1913, 'blogpost': 573, 'swiftly': 4291, 'sharpens': 3946, 'fang': 1603, 'radicalisation': 3573, 'resignation': 3708, 'carlpaladino': 758, 'jeff': 2361, 'bustyescos': 693, 'attendants': 366, 'genderequity': 1811, 'journos': 2398, 'remark': 3675, 'uttered': 4659, 'kingfisher': 2449, 'antiracism': 276, 'seashepherd': 3876, 'condescension': 970, 'verbally': 4680, 'assailed': 345, 'hmmmsounds': 2084, 'familiar': 1596, 'mongers': 2872, 'loudest': 2644, 'malia': 2701, 'finepass': 1678, 'gtfoh': 1930, 'itz': 2344, 'crackalogic': 1062, 'malign': 2702, 'arresting': 326, 'choked': 845, 'gentlemen': 1820, 'neoliberalism': 2986, 'prioritized': 3443, 'cochair': 907, 'michelle': 2808, 'p21': 3201, 'foretold': 1726, 'sweep': 4288, 'teambts': 4338, 'broad': 657, 'wyou': 4939, 'judged': 2403, 'nelsonmandela': 2984, 'ra': 3556, '2017fail': 35, 'originally': 3172, 'landholding': 2487, 'males': 2699, '2016in4words': 32, 'rightie': 3753, 'refugee': 3643, 'sikh': 3998, 'vandalised': 4666, 'wso': 4930, 'condemns': 969, '99c99p': 81, 'dystopian': 1378, 'yesu': 4962, 'skinned': 4028, 'kwanzaa': 2475, 'whitesupremacists': 4837, 'happykwanzaa': 1972, 'happyholidays': 1971, 'merrychristmas': 2793, 'africanamerican': 162, 'reparations': 3685, 'restricted': 3721, 'straght': 4207, 'jacket': 2349, 'thatblacklivesmatter': 4386, 'amjoy': 235, 'biden': 507, 'unmasking': 4627, 'americas': 232, 'stripe': 4219, 'misunderstood': 2851, 'trumpracist': 4545, 'trumpconaist': 4540, 'scatter': 3852, 'blessampkeep': 559, 'gracious': 1891, 'despise': 1233, 'littleenglander': 2603, 'sovereignty': 4122, 'kumbaya': 2473, 'antisemitism': 280, 'exemplifies': 1535, 'swamp': 4284, 'draining': 1339, '20days': 37, 'taxpayer': 4329, 'islamicterrorist': 2325, 'enabler': 1441, 'trough': 4533, 'piggy': 3306, 'michelleobama': 2809, 'thug': 4440, 'agitator': 172, 'stl': 4190, 'universities': 4620, 'expats': 1542, 'privilege': 3447, 'endorse': 1450, 'inauguration': 2217, 'opkillingbay': 3148, 'helpcovedolphins': 2035, 'thecove': 4393, 'redcove': 3634, 'misogynist': 2842, 'voterid': 4720, 'kentucky': 2423, 'jcpenny': 2359, 'bitchesbelike': 534, 'africanamericans': 163, 'jeffsessions': 2363, 'imagemakeovers': 2196, 'makeo': 2691, 'intolerant': 2300, 'postbrexit': 3386, 'hatecrime': 1988, 'compensatory': 954, 'callin': 714, '6417153640': 69, '564943': 65, 'iswar': 2338, 'merocrush': 2791, 'cathedral': 768, 'xenophobia': 4942, 'komo': 2468, 'jonjo': 2391, 'shelvey': 3950, 'racially': 3566, 'kickitout': 2432, 'amrica': 239, 'abe': 86, 'mere': 2788, 'rightist': 3754, 'nytimes': 3085, 'rsf': 3797, 'alexjones': 194, 'draintheswamp': 1340, 'idiocy': 2172, 'microaggressions': 2811, 'ag': 166, 'scrubs': 3868, 'recordeven': 3630, 'argued': 317, 'trumpsamerica': 4547, 'trumpworld': 4551, 'euroconservatism': 1499, '1000x': 4, 'wud': 4933, 'blackonblack': 545, 'ris2016': 3760, 'antiblackness': 272, 'dogwhistle': 1315, 'hotline': 2121, 'jumpsuit': 2406, 'subtle': 4239, 'buttoning': 698, 'riyadh': 3763, 'renowned': 3684, 'thugs': 4441, 'jail': 2351, 'peotus': 3266, 'opponent': 3150, 'germans': 1825, 'fascists': 1612, 'bluelivesmatter': 579, 'taragon': 4320, 'christmassy': 859, 'herb': 2040, 'sexist': 3928, 'tuckercarlson': 4557, 'crashed': 1067, 'greenspan': 1909, 'ark': 320, 'discoved': 1276, 'errand': 1481, 'hoodiesup': 2107, 'refugeeswelcome': 3645, 'stlouis': 4191, 'muthafucka': 2931, 'unleashed': 4623, 'attorneygeneral': 371, 'racewar': 3564, 'weaponry': 4775, 'saleh': 3822, 'wwwsmallgirlsexcom': 4938, 'economicsanctions': 1397, 'whiteprivilege': 4833, 'roughwaters': 3791, 'jc': 2358, 'penney': 3261, 'headlock': 2013, 'corporatebs': 1026, 'boycottjcpenney': 623, 'educating': 1405, 'systemic': 4300, 'goaded': 1856, 'inmates': 2258, 'restrained': 3720, 'offended': 3105, 'phrases': 3296, 'sandniggers': 3828, 'cab': 704, 'islamaphobia': 2321, 'professor': 3464, 'jews': 2369, 'simpleton': 4005, 'overrode': 3194, 'trc': 4511, 'stereotypes': 4187, 'snowflakes': 4075, 'remote': 3681, 'justjustice': 2412, 'thanksjackasses': 4383, 'packyourshitobama': 3206, 'inclusivity': 2224, 'tra': 4495, 'deltaairlines': 1198, 'palestine': 3215, 'antiwhite': 281, 'panthers': 3220, 'milwaukee': 2824, 'chant': 810, 'cracker': 1063, 'spew': 4137, 'trumpism': 4541, 'intrumpsamerica': 2301, 'extending': 1559, 'uniting': 4619, 'hypocrisy': 2156, 'manypols': 2715, 'ferguson': 1656, 'priyankachopra': 3448, 'subjected': 4236, 'quantico': 3539, 'xians': 4944, 'definitive': 1188, 'ushistory': 4651, 'afamhist': 151, 'moron': 2889, 'destructive': 1240, 'socialists': 4080, 'maddow': 2676, 'correct': 1028, 'profession': 3463, 'diagram': 1248, 'theresistance': 4406, 'resist': 3710, 'overexcited': 3192, 'fearlessly': 1632, 'propalestine': 3485, 'antisemites': 279, 'retard': 3725, 'kaepernick': 2416, 'whitey': 4840, 'griffiths': 1914, 'pattismith': 3245, 'reich': 3656, 'jeffery': 2362, 'beauregard': 454, 'foolin': 1719, 'sheets': 3949, 'saturda': 3836, 'coal': 906, 'electrical': 1419, 'producers': 3458, 'dispose': 1292, 'sulfur': 4250, 'stockpiles': 4193, 'cologne': 919, '2016in4worlds': 33, 'gwot': 1945, 'boycottdelta': 622, 'adamsaleh': 132, 'prankster': 3406, 'atwater': 374, 'boob': 601, 'camron': 727, 'investigate': 2305, 'prosecute': 3490, 'aptly': 309, 'ndn': 2968, 'tyler': 4582, 'perry': 3277, 'backlash': 403, 'reverse': 3736, 'establishment': 1486, 'blk': 565, 'folx': 1713, 'stab': 4157, 'standwithisrael': 4169, 'obamajarretts': 3087, 'inflict': 2246, 'wonthey': 4894, 'fuhered': 1777, 'declared': 1169, 'putinspuppet': 3535, 'spiced': 4138, 'reefers': 3635, 'negro': 2980, 'anslinger': 267, 'swingtanzenverboten': 4295, 'rapacious': 3591, 'sealed': 3874, 'doom': 1328, 'teamsuperjunior': 4341, 'bandying': 419, 'discrimination': 1279, 'skynews': 4030, 'presenter': 3422, 'aampe': 83, 'cjreform': 868, 'deathpenalty': 1160, 'juvenilejustice': 2414, 'bailreform': 411, 'prosecutorialmisconduct': 3492, 'cora': 1023, 'rebellion': 3623, 'dontbuythesun': 1327, 'rags': 3578, 'tories': 4479, 'dull': 1369, 'nigelfarage': 3016, 'swastika': 4286, 'whitepower': 4830, 'feminismiscancer': 1651, 'feminismisterrorism': 1652, 'feminismmuktbharat': 1653, 'malevote': 2700, 'chaplin': 812, 'dictator': 1251, 'mayor': 2748, 'extremist': 1565, 'emiratis': 1428, 'messi': 2795, 'cr7': 1060, 'meccalive': 2762, 'leftright': 2546, 'polarisation': 3351, 'uselections2016': 4646, 'charles': 816, 'paladinos': 3214, 'mrminority': 2910, 'normalizing': 3051, 'betrays': 494, 'activism': 121, 'nofur': 3032, 'animalrights': 255, 'noyulin': 3068, 'wakemeupwhen': 4738, 'stops': 4202, 'liberallogic': 2569, 'bhakt': 504, 'cornered': 1024, 'rational': 3602, 'charlespaladinos': 817, 'wailed': 4731, 'sundown': 4259, 'towns': 4493, 'route66': 3795, 'travellers': 4510, 'segregation': 3893, 'darkest': 1138, 'moviefone': 2905, 'postholiday': 3389, 'nowthattheholidaysaredone': 3067, 'hating': 1996, '12mill': 10, 'arrests60mill': 328, 'policecivilian': 3354, 'usshootingsaccidentsabuse': 4655, 'rarethous': 3597, 'womens': 4884, 'germanys': 1827, 'cease': 785, 'priviledge': 3446, 'shitler': 3960, 'ignorantly': 2188, 'bleating': 557, 'projecting': 3476, 'palladino': 3217, 'forbidden': 1722, 'dzange': 1379, 'capetown': 746, 'republicantrash': 3697, 'spit': 4144, 'meditations': 2767, 'foxnews': 1740, 'underfire': 4606, 'treason': 4514, 'aidingenemies': 181, 'coverups': 1052, 'benghazi': 484, 'extoion17': 1561, 'marathi': 2716, 'lds': 2516, 'stands': 4168, 'adultery': 145, 'incest': 2219, 'pedophilia': 3256, 'props': 3489, 'biggotry': 511, 'vile': 4698, 'willoughbys': 4856, 'entitlement': 1470, 'socialized': 4081, 'innate': 2260, 'revoltingly': 3739, 'entitled': 1469, 'tws': 4580, 'hatespeech': 1995, 'psa': 3510, 'ethnicity': 1491, 'biblical': 506, 'overlap': 3193, '1117': 7, 'feminists': 1655, 'materials': 2739, 'tyruswong': 4587, 'bambi': 416, 'thwaed': 4444, 'constrained': 999, 'adl': 137, 'philandocastile': 3287, 'trumpusa': 4550, 'walks': 4745, 'refuses': 3646, 'reject': 3657, 'substantive': 4238, 'echo': 1394, 'irritations': 2317, 'newghostbusters': 3001, 'indisputably': 2239, 'intrusively': 2302, 'imitations': 2199, 'whiners': 4821, 'mmusimaimane': 2855, 'malema': 2698, 'combined': 928, 'policebrutality': 3353, 'coherently': 912, 'unloads': 4626, 'horrifying': 2115, 'anc': 246, 'neogaf': 2985, 'kkkblm': 2457, 'hitlerjugend': 2080, 'rife': 3751, 'reginald': 3650, 'mcfadden': 2752, 'interviews': 2297, 'returningcitizens': 3728, 'commutations': 949, 'defunding': 1189, 'aboion': 92, 'stable': 4158, 'homosex': 2103, 'cliches': 883, 'semantics': 3900, 'ageism': 170, 'sob': 4077, 'plaster': 3326, 'lettuce': 2556, 'maters': 2740, 'slices': 4044, 'mispronouncing': 2846, 'terraced': 4365, 'justified': 2411, 'apologize': 297, 'probab': 3449, 'freedownload': 1752, 'holier': 2094, 'moi': 2862, 'hangs': 1962, 'capitol': 748, 'fetishes': 1659, 'harmless': 1982, 'objectification': 3091, 'bondage': 594, 'michigan': 2810, 'sedition': 3886, 'i95n': 2159, 'stain': 4163, 'mormon': 2885, 'megyn': 2772, 'fauxnews': 1622, 'santa': 3830, 'universityofwisconsinmadison': 4622, 'problemofwhiteness': 3453, 'orgasmic': 3170, 'intersectional': 2295, 'whitefeminism': 4824, 'ableism': 91, 'gu': 1932, 'openly': 3145, 'reprehensible': 3692, 'whitesupremacist': 4836, 'depl': 1216, 'terming': 4363, 'denounce': 1214, 'combat': 927, 'misogynistic': 2843, 'falsenarrative': 1595, 'falseequavalency': 1594, 'obstacles': 3094, 'mocked': 2858, 'trafficking': 4499, 'scapegoat': 3847, 'nc': 2967, 'lawmakers': 2511, 'surgical': 4274, 'precision': 3414, 'assistants': 350, 'gladshesgone': 1849, 'nohope': 3034, 'ovr': 3197, 'uspoli': 4654, 'gt15': 1927, 'versace': 4681, '142017': 16, 'sourcenation': 4117, 'thependulum': 4400, 'newshow': 3004, 'vp': 4723, 'protest': 3498, 'demagoguery': 1201, 'obstruction': 3095, 'liz': 2609, 'treacy': 4513, 'condone': 974, 'excouncillor': 1531, 'hacketts': 1948, 'islamophobia': 2328, 'lede': 2539, 'associates': 351, 'approval': 306, 'riddance': 3747, 'zionism': 4995, 'israelisettlements': 2334, 'woow': 4900, 'lumpy': 2666, 'bwahahahahaha': 700, 'overwhelmingly': 3196, 'socially': 4083, 'bewitched': 500, 'mizzou': 2853, 'cam': 718, 'megaupload': 2771, 'oust': 3183, 'jabs': 2348, 'ironies': 2315, 'inherent': 2252, 'barackobama': 426, 'documented': 1310, 'nazis': 2963, 'brownshi': 666, 'pivo': 3314, 'complained': 958, 'pap': 3222, 'affirmativeaction': 155, 'quotas': 3551, 'insecurity': 2266, 'crapsac': 1066, 'techjunkiejhaltright': 4346, 'deplorable': 1217, 'savor': 3841, 'quebec': 3540, 'opkillingbaythecove': 3149, 'colonialism': 921, 'imperialism': 2207, 'brainsjust': 631, 'eyesvote': 1569, 'lunatics': 2668, 'henrico': 2037, 'vandals': 4667, 'dec': 1164, '946': 78, 'asssssworst': 352, 'everlord': 1509, 'gt6': 1929, 'victimise': 4689, 'oppressor': 3157, 'gt10': 1926, 'malayalees': 2695, 'picturebangladeshi': 3302, 'defile': 1181, 'maimane': 2685, 'thatzille': 4389, 'himhe': 2064, 'shou': 3974, 'effect': 1408, 'xenopbobia': 4941, 'conflate': 982, 'matthew': 2744, 'immanuel': 2200, 'whic': 4817, 'cspan': 1102, 'alternet': 213, 'regressive': 3653, 'wout': 4921, 'slurs': 4048, 'refuted': 3647, 'sunderland': 4258, 'buddhist': 674, 'burmese': 685, 'buddhism': 673, 'rohingyas': 3772, 'neighborhoods': 2982, 'muzzies': 2932, 'veiled': 4674, 'blackface': 542, 'mummersparade': 2922, 'tens': 4361, 'candid': 734, 'sexting': 3930, 'condoning': 975, 'fakenewsale': 1591, 'hated': 1989, 'crookedtrump': 1093, 'rother': 3787, 'sue': 4246, 'warmonger': 4756, 'worlddanger': 4912, 'communist': 946, 'gfy': 1832, 'highlycontested': 2057, 'profitable': 3468, 'bigoted': 513, 'notfit': 3056, '99c': 80, '99p': 82, 'religions': 3671, 'unifying': 4613, 'nobdy': 3028, 'conaist': 964, 'islamaphobic': 2322, 'russianhack': 3808, 'nomandate': 3037, 'colinpowell': 914, 'condoleezzarice': 972, 'secretaryofstate': 3884, 'josephjett': 2393, 'iconic': 2165, 'batshit': 438, 'sjws': 4026, 'notracist': 3064, 'parentage': 3225, 'bernanke': 486, 'bailed': 409, 'dumping': 1371, 'genetics': 1816, 'mt': 2915, 'help700': 2034, 'povey': 3396, '1625': 21, 'nationala': 2953, '13479': 12, 'stupids': 4233, 'netanyahu': 2991, 'tube': 4556, 'conflates': 983, 'define': 1182, 'systemicracism': 4301, 'ushers': 4650, 'incineration': 2221, 'asians': 339, 'rooted': 3783, 'societys': 4086, 'candide': 737, 'uyanze': 4660, 'cult': 1106, 'requires': 3703, 'patriotwatch': 3244, '148': 17, 'projection': 3477, 'auspol': 379, 'upf': 4636, 'cultism': 1107, 'springst': 4152, 'ape': 294, 'attracted': 373, 'sends': 3904, 'plethora': 3337, 'fashio': 1613, 'notmypres': 3060, 'notmypresident': 3061, 'occurredjust': 3098, 'marxism': 2729, 'swampmonster': 4285, 'engage': 1458, 'aberrant': 88, 'treasonoustrump': 4515, 'whiteisis': 4827, 'nuclear': 3072, 'homophobe': 2100, 'regularass': 3655, 'protects': 3497, 'vps': 4724, 'multimillion': 2919, 'bonuses': 599, 'endof2016': 1449, 'tedtalks': 4349, 'tedatibm': 4348, 'pandy': 3219, 'phillysuppophilly': 3288, 'enduring': 1453, 'conviction': 1012, 'korematsu': 2471, 'quest': 3543, 'humanrights': 2143, 'cambersands': 720, 'lifeguards': 2578, 'gt2': 1928, 'secretly': 3885, 'sunbury': 4255, 'fliers': 1696, 'mosque': 2890, 'techjunkiejhthe': 4347, 'unpacks': 4628, 'blackwomen': 551, 'asianmen': 338, 'datingapps': 1145, 'omits': 3130, 'weasel': 4778, 'appeared': 302, 'nds': 2969, 'beloy': 482, 'varying': 4669, 'swastikas': 4287, 'crosses': 1095, 'neonazi': 2987, 'eugenics': 1494, 'threaten': 4431, 'boycotts': 624, 'yiannopoulos': 4964, 'need2know': 2975, 'clintons': 890, 'drinkdark': 1350, 'biracial': 526, 'braids': 629, 'jolly': 2390, 'consumers': 1001, '2016in': 31, 'afl': 157, 'tattler': 4324, 'janeelliot': 2353, 'salt': 3824, 'tribune': 4525, 'betwn': 497, 'plannedparenthood': 3323, 'freedoms': 1751, 'robbiewilliams': 3768, 'lid': 2573, 'selfhating': 3896, 'libearian': 2567, 'cdnpoli': 783, 'canpoli': 741, 'aande': 84, 'resource': 3713, 'inuit': 2303, 'axed': 395, 'cbc2017': 780, 'politicalmillennial': 3361, 'datesletstalkabout': 1143, 'climatechange': 886, 'valuechain': 4664, 'hugh': 2138, 'muir': 2918, 'jind': 2374, 'insident': 2269, 'qualifications': 3537, 'bozzoli': 627, 'highereducation': 2054, 'hires': 2071, 'mfer': 2803, 'thegreenpalmcottage': 4394, 'plettenburgbay': 3338, 'westerncape': 4805, 'thyini': 4446, 'thiza': 4421, 'basket': 433, '50s': 64, '230pmet': 40, 'twisted': 4573, 'thatmakes': 4387, 'nutshell': 3080, 'bragged': 628, 'grabbing': 1888, 'pussies': 3528, 'koreans': 2470, 'joseon': 2392, 'demo': 1205, 'aberdeen': 87, 'humiliated': 2145, 'mdcollege': 2753, 'deploying': 1218, 'interfering': 2290, 'hardcoresex': 1977, 'movements': 2903, 'philosopher': 3289, 'aleksandr': 193, 'dugin': 1366, 'pastors': 3238, 'nye': 3083, 'rapeculture': 3593, 'islamofascist': 2326, 'rohingya': 3771, 'officials': 3117, 'myanmars': 2934, 'rul': 3800, 'refresher': 3642, 'loveknow': 2650, 'indianapolis': 2234, 'colts': 925, 'cheerleader': 824, 'slur': 4047, 'straya': 4213, 'factophobia': 1583, 'culturalenrichment': 1108, 'snowball': 4073, 'demonic': 1210, 'replies': 3687, 'greenspans': 1910, 'smug': 4067, 'annihilated': 260, 'sara': 3831, 'tweet4taiji': 4568, 'supermistict': 4265, 'douchebag': 1334, 'poke': 3348, 'sme': 4055, 'burninhell': 688, 'sjwlogic': 4025, 'messiah': 2796, '1996': 24, 'zionists': 4997, 'hebrews': 2024, '2017in3words': 36, 'mma': 2854, 'doublestandards': 1333, 'smileyong': 4061, 'kingconrad': 2447, '75': 73, 'empowerd': 1437, 'fowoh': 1738, 'arrest': 324, 'nunes': 3076, 'womenmalayalam': 4882, 'ameture': 234, 'nonracist': 3043, 'military': 2817, 'putins': 3533, 'commie': 940, 'deafening': 1155, 'antiracist': 277, 'happynewyear': 1973, 'genderinequality': 1812, 'imo': 2205, 'metaphor': 2798, 'strident': 4218, 'ordination': 3166, 'womensordination': 4886, '642': 70, 'irrefutable': 2316, 'truism': 4537, 'tolerated': 4468, 'supremacy': 4272, 'freemasons': 1754, 'dependent': 1215, 'crimes': 1087, 'politicalcorrectness': 3359, 'salutbut': 3825, 'sharp': 3945, 'richardspencer': 3744, 'nojustice': 3036, 'loserpizza': 2635, 'infidel': 2244, 'childabuse': 832, 'backhanders': 402, 'invisible': 2306, 'championed': 804, 'donaldtrumps': 1322, 'unleashes': 4624, 'feared': 1631, 'probation': 3451, 'blackperson': 547, 'districts': 1298, 'birdsofafeather': 529, 'pt': 3512, 'evolutions': 1521, 'relations': 3660, 'welleducated': 4798, 'destigmatize': 1236, 'gweh': 1944, 'wypipo': 4940, 'slaver': 4038, 'cretin': 1083, 'carlpaladinoelection': 759, 'cochairman': 908, 'presidentelect': 3424, 'mrampmrs': 2909, 'trht': 4523, 'spinning': 4140, 'fuckdonaldtrump': 1770, '13thdocumentary': 14, 'concur': 967, 'aryanbrotherhood': 332, 'onlywhitechristmas': 3138, 'drsuess': 1358, 'moonman': 2880, 'edgy': 1400, 'dankmemes': 1135, 'realitycheck': 3617, 'policing': 3356, 'audiblechannels': 375, 'scapelliti': 3848, 'woobietuesday': 4895, 'prolapsed': 3478, 'prick': 3436, 'greentown': 1911, 'hocoschools': 2086, 'columbiamd': 926, 'hocomd': 2085, 'ahole': 177, 'extraordinaire': 1562, 'fascismo': 1610, 'antiwhites': 282, 'clio': 891, 'twitte': 4575, 'obsidian': 3093, 'latesnews': 2499, 'twitterbot': 4577, 'genocidedefender': 1819, 'ai': 178, 'islamichomophobicrapistwho': 2324, 'tremendous': 4519, 'corps': 1027, 'poopin': 3367, 'cussed': 1116, 'attire': 368, 'dreads': 1343, 'shabbatshalom': 3935, 'fucktrump': 1773, 'sexualpredator': 3933, 'incompitent': 2225, 'igmilitia': 2184, 'pewpewlife': 3285, 'historian': 2074, 'whitewashing': 4839, 'ukraines': 4596, 'volodymyrviatrovych': 4714, 'repugnant': 3698, 'pussygrabberinchief': 3530, 'idiocracy': 2171, 'marxist': 2730, 'propoganda': 3488, 'cite': 862, 'grou': 1917, 'porns': 3376, 'dkpol': 1305, 'levin': 2559, 'frances': 1744, 'cress': 1082, 'welsing': 4800, 'drfranciscresswelsing': 1347, 'mafuckas': 2680, 'contempt': 1002, 'activist': 122, 'wus': 4934, 'threatened': 4432, 'founder': 1736, 'defines': 1184, 'oppressorsprivileged': 3158, 'insanity': 2264, 'wwi': 4936, 'wwii': 4937, 'waged': 4730, 'resigned': 3709, 'justicereform': 2410, 'judges': 2404, 'coworker': 1056, 'goverment': 1883, 'womensrighttochoose': 4887, 'hamzayusuf': 1956, 'sociopathic': 4088, 'fled': 1695, 'fugly': 1775, 'lunatic': 2667, 'neverdelta': 2997, 'resent': 3706, 'evolution': 1520, 'attorneys': 372, 'whopped': 4844, 'ronda': 3777, 'rousey': 3793, 'definitions': 1187, 'disabilities': 1271, 'ivankatrump': 2346, 'manspreading': 2712, 'camber': 719, 'mtv': 2916, 'wtf2016': 4932, 'unpresidented': 4629, 'timing': 4454, 'zionist': 4996, 'whitenationalist': 4828, 'slander': 4033, 'dictionary': 1252, 'averse': 388, 'appropriating': 305, 'ultimately': 4598, 'macdonald': 2673, 'ethnocentrism': 1492, 'pegida': 3258, 'teammichaelpalage': 4340, 'teamcancer': 4339, 'kitazatoshibasaburo': 2453, 'tetanusimmunization': 4373, 'kitazato': 2452, 'didnot': 1254, 'nobelprize': 3029, 'literacy': 2600, 'adultedu': 144, 'repulsion': 3699, 'desirehatred': 1231, 'herschlags': 2046, 'haim': 1952, 'yavin': 4951, 'cooke': 1015, 'ridiculously': 3749, 'pavillions': 3247, 'procrastinate': 3456, 'shitshow': 3961, '2stand': 50, 'highstool': 2058, '2raise': 49, 'servant': 3915, 'mormonism': 2887, 'mormonis': 2886, '940pm': 77, 'southsudankot': 4121, 'intermittently': 2291, 'lolol': 2620, 'leb': 2535, 'busily': 691, 'racemix': 3562, 'trumpleaks': 4542, 'misogynisttrump': 2844, 'catdjt': 767, 'impeachtrump': 2206, 'ating': 359, 'jewish': 2368, 'australians': 383, 'whitepriv': 4831, 'shuckinandjivin': 3986, 'democraticpay': 1208, 'westerners': 4806, 'islamophobes': 2327, 'apostolis': 298, 'theresamay': 4405, 'remainers': 3673, 'empress': 1438, 'regnant': 3652, 'distant': 1295, 'minoritypresident': 2836, 'stopthewars': 4203, 'disavow': 1273, 'nonsensical': 3044, 'knicks': 2460, 'praised': 3404, 'consent': 992, 'decree': 1172, 'tamirrice': 4317, 'kellyann': 2422, 'skywilliams': 4031, 'wingers': 4862, 'bluntly': 581, 'richie': 3745, 'toures': 4489, 'coronial': 1025, 'inquest': 2262, 'hamids': 1955, 'verbalassault': 4679, 'waime': 4732, 'wmn': 4876, 'whiter': 4834, 'federline': 1634, 'serves': 3917, 'hitler': 2079, 'collectively': 917, 'jemele': 2364, 'zionazi': 4993, 'amerikkka': 233, 'demonstrated': 1211, 'disrespect': 1293, 'feigned': 1643, 'excrement': 1532, 'ofc': 3103, 'cameronstaff': 723, 'camstaff': 728, 'officerstaff': 3114, 'acab': 103, 'cowardcops': 1054, 'fightback': 1664, 'peacein2017': 3254, 'latent': 2496, 'melted': 2776, 'snowflake': 4074, 'heed': 2026, 'triggerwarning': 4529, 'slant': 4034, 'representation': 3694, 'communism': 945, 'fp': 1741, 'gegenrechts': 1808, 'nostrache': 3053, 'noafd': 3027, 'nopegida': 3048, 'populism': 3374, 'nonazis': 3041, 'aufstehn': 376, 'discrepancy': 1277, 'rare': 3596, 'pissing': 3312, 'quebecs': 3541, 'pizzagate': 3316, 'commentsyou': 937, 'barack': 425, 'ethnic': 1490, 'girles': 1841, 'smth': 4066, 'gaslighting': 1800, 'detroit': 1243, 'pamelaramseytaylorpraised': 3218, 'beverlywhalingreinstated': 498, 'stateampfederal': 4176, 'funde': 1782, 'dipshit': 1267, 'opendemocracy': 3143, 'blacklivesmatters': 544, 'lanasprayberry': 2485, 'authority': 385, 'blacktwitter': 549, '342': 53, 'nooltham': 3045, 'eltham': 1423, 'asylumseekers': 356, 'saturdayblogshare': 3838, 'racecard': 3561, 'trumpproofamerica': 4544, 'manning': 2711, 'rushlimbaugh': 3805, 'enlightenment': 1465, 'wilfully': 4853, 'miloyiannopoulos': 2823, 'womenrelated': 4883, 'womensissues': 4885, 'whitefish': 4825, 'blackbrown': 541, 'communities': 947, 'doublestandard': 1332, 'niga': 3015, 'corrupttrump': 1032, 'sexoffender': 3929, 'notmyprez': 3063, 'taxevader': 4328, 'facemask': 1578, 'unintentionally': 4614, 'moredun': 2882, 'prosecuted': 3491, 'depoation': 1219, 'authoritarian': 384, 'jacobmarley': 2350, 'narrow': 2949, 'tlot': 4460, 'sdob': 3871, 'shiless': 3955, 'unabated': 4601, 'obamanation': 3088, 'antiegalitarianism': 274, 'narcissism': 2946, 'chauvinist': 820, 'lostpopularvote': 2641, 'americadoesntwantyou': 229, 'ksa': 2472, 'flood': 1699, '80snostalgia': 74, 'gollybar': 1865, 'nbavote': 2966, 'popularvote': 3372, 'strangely': 4209, 'twonitwittrumpsuppoers': 4579, 'hiv': 2082, 'infection': 2243, 'oral': 3161, 'ampamp': 238, 'arise': 319, 'farah': 1607, 'islamophobic': 2329, 'ifyourenotwhiteyourenotracist': 2181, 'instructed': 2281, 'races': 3563, 'brutality': 668, 'readingbuying': 3611, 'rector': 3632, 'radicalism': 3574, 'woodrow': 4897, 'wilsons': 4857, 'trendy': 4521, 'scalds': 3846, 'aches': 114, 'chokes': 846, 'smog': 4063, 'classism': 876, 'imperialsm': 2208, 'what2inlieuof': 4811, 'narcissimmisogyny': 2945, 'whate': 4812, 'repulsive': 3700, 'kurds': 2474, 'journo': 2397, 'claimed': 870, 'antics': 273, 'purposedly': 3525, 'wakeupamericaca': 4740, 'phenomenon': 3286, 'reemergence': 3636, 'discourse': 1275, 'cityjournal': 866, 'surrogate': 4277, 'cannon': 739, 'alllivesmatter': 201, 'unbornlivesmatter': 4603, 'convo': 1014, 'epidemic': 1476, 'atsi': 361, 'niggers': 3021, 'compared': 951, 'roughly': 3790, '400000': 59, 'invasion': 2304, 'jlaw': 2375, 'expecttrumps': 1547, 'racismfacinghis': 3569, 'leftistlunatics': 2545, 'powerhungrytraitors': 3399, 'hereticfoundation': 2044, 'catholicchurch': 769, 'carlpaladinos': 760, 'loathsome': 2612, 'illegals': 2193, 'andor': 249, 'waiter': 4734, 'himselffor': 2066, 'sakesstop': 3819, 'satire': 3835, 'breeds': 642, 'prejudice': 3415, 'overused': 3195, 'drill': 1348, 'pridemake': 3438, 'enjoyable': 1463, 'informing': 2251, 'laughable': 2504, 'snaps': 4070, 'womenvote': 4888, 'statementdoubling': 4178, 'mediamy': 2764, 'avoice': 389, 'libcrib': 2566, 'negligence': 2979, 'exists': 1540, 'oldwoman': 3127, 'shadowed': 3937, 'wellduh': 4797, 'manus': 2713, 'sexualharassment': 3932, 'laborviolations': 2478, 'yoyas': 4983, 'deming': 1204, 'comparisons': 952, 'childish': 834, 'pornstars': 3377, 'racebaiting': 3560, 'letstalkabout': 2553, 'thepoliticalmillennial': 4401, 'embrac': 1426, 'jibes': 2370, 'gravity': 1902, 'opposed': 3152, 'climatecrimes': 887, 'ecocide': 1395, 'blackvotehispanicvote': 550, 'warsforoil': 4759, 'yehtut': 4957, 'humiliating': 2146, 'tacky': 4304, 'notmypresidenttheresista': 3062, 'cept': 797, 'symbols': 4296, 'couesy': 1034, 'galery': 1790, 'vestal': 4684, 'pupil': 3520, 'suing': 4248, 'district': 1297, 'waitingf': 4736, 'wildest': 4851, 'takuyakimura': 4310, 'highfashion': 2056, 'progresverebel': 3473, 'malvern': 2705, 'helm': 2032, 'corruptgop': 1030, 'womenwednesday': 4889, 'starspangledbanner': 4173, 'pedophiles': 3255, 'satanists': 3834, 'bonhoeffer': 597, 'kindledeals': 2443, 'aswell': 355, 'allies': 199, 'unstoppable': 4631, 'christmaseve': 858, 'ivanka': 2345, 'crusader': 1099, 'sequel': 3911, 'hvg': 2154, 'fibromyalgia': 1662, 'childrape': 836, 'native': 2957, 'jarodtaylor': 2357, 'baycat': 440, '2nites': 48, '4maps': 61, 'disciples': 1274, 'carlpalidino': 761, 'vomit': 4716, 'truckload': 4535, 'firstfamily': 1685, 'gandhi': 1796, 'adores': 141, 'nominate': 3038, 'muslimbrotherhood': 2927, 'antimorality': 275, 'contradictions': 1007, 'iefascistdemocracy': 2179, 'largely': 2489, 'mogage': 2861, 'defenders': 1178, 'russias': 3809, 'graceless': 1890, 'suppoed': 4267, 'devil': 1246, 'katiesikphoto': 2417, 'thesexismproject': 4408, 'fabricate': 1573, 'swinging': 4294, 'countered': 1041, 'inhumans': 2254, 'pennyloafer': 3263, 'malikrichmond': 2703, 'trentmays': 4522, 'prohibition': 3474, 'utah': 4658, 'inflame': 2245, 'derek': 1224, 'itstheendoftheworldand': 2343, 'barrackobama': 427}

    words = review_to_words(event['body'])
    bow = bow_encoding(words, vocab)

    # The SageMaker runtime is what allows us to invoke the endpoint that we've created.
    runtime = boto3.Session().client('sagemaker-runtime')

    # Now we use the SageMaker runtime to invoke our endpoint, sending the review we were given
    response = runtime.invoke_endpoint(EndpointName = 'twitter-sentiment-endpoint',# The name of the endpoint we created
                                       ContentType = 'text/csv',                 # The data format that is expected
                                       Body = ','.join([str(val) for val in bow]).encode('utf-8')) # The actual review

    # The response is an HTTP response whose body contains the result of our inference
    result = response['Body'].read().decode('utf-8')

    # Round the result so that our web app only gets '1' or '0' as a response.
    result = round(float(result))

    return {
        'statusCode' : 200,
        'headers' : { 'Content-Type' : 'text/plain', 'Access-Control-Allow-Origin' : '*' },
        'body' : str(result)
    }

In [21]:
print(xgb_combined_predictor.endpoint_name)
print("\n",vocabulary)

twitter-sentiment-endpoint

 {'just': 2408, 'finished': 1681, 'with': 4872, 'awesome': 393, 'amp': 237, 'super': 4263, 'staff': 4160, 'im': 2195, 'to': 4461, 'work': 4903, 'together': 4465, 'la': 2477, 'angry': 253, 'birds': 528, 'film': 1668, 'download': 1336, '2016': 30, 'my': 2933, 'new': 2999, 'book': 602, 'cover': 1049, 'so': 4076, 'much': 2917, 'books': 604, 'romance': 3775, 'preorder': 3417, 'love': 2646, 'twitch': 4574, 'instead': 2280, 'in': 2216, 'but': 695, 'international': 2292, 'debate': 1163, 'on': 3131, 'the': 4390, 'best': 488, 'kind': 2440, 'of': 3102, 'happy': 1970, 'friday': 1757, 'yall': 4950, 'mood': 2878, 'funny': 1786, 'instadaily': 2274, 'instamood': 2278, 'weekend': 4788, 'training': 4503, 'fitfam': 1688, 'fitness': 1689, 'tgif': 4376, 'cant': 742, 'wait': 4733, 'go': 1855, 'see': 3887, 'finding': 1675, 'shot': 3973, 'this': 4420, 'months': 2877, 'issue': 2335, 'almost': 207, 'its': 2341, 'quite': 3550, 'plain': 3319, 'and': 248, 'simple': 4004, 'it': 2339, 'wo